# Converting Homestuck Collection data to an agnostic dataset

This notebook has all the steps for taking data from the [Unofficial Homestuck Collection](https://bambosh.dev/unofficial-homestuck-collection/)'s asset pack and converting it into a dataset that can be used for training a machine learning model.

Some planned uses for this are:
- Summarization
- Style transfer/LORAs
- Chatbots

# Constants and Imports

In [2]:
"""
IMPORTS
Put all at the beginning because I hate notebooks so much
"""
import json
import os
import re

import pandas as pd
import requests
from IPython.display import Image
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI

In [3]:

"""
CONSTANTS
This will extract constants from the env variables set in the .env file 
and make them accessible to the notebook
"""
print("Loading variables from .env file\n...")
load_dotenv()

ASSET_PACK_FOLDER = os.getenv("ASSET_PACK_FOLDER")
OUTPUT_FOLDER = os.getenv("OUTPUT_FOLDER")
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
MODEL_ID = os.getenv("MODEL_ID")
print("Loaded variables successfully")

print("Loading constants\n...")
# Relevant folders and files

# Bespoke input files that aren't available elsewhere
INPUT_FOLDER = os.path.join(os.path.dirname(os.path.abspath('')), "input")

# Transcripts and commentary from ReadMSPA, assembled by Bambosh, Makin and Giovanh
MSPA_COMMENTARY = os.path.join(INPUT_FOLDER, "commentary.json")

# Full transcripts from ReadMSPA, assembled by Giovanh and Bambosh
READMSPA_ALTNARRATIVE = os.path.join(INPUT_FOLDER, "altnarrative.json")
READMSPA_ALTTEXT = os.path.join(INPUT_FOLDER, "alttext.json")
READMSPA_TRANSCRIPTS = os.path.join(INPUT_FOLDER, "transcripts.json")

# Panel tags from the Homestuck Search Engine
HSSE_TAGS = os.path.join(INPUT_FOLDER, "hsse_tags.json")
HSSE_SEARCH = os.path.join(INPUT_FOLDER, "hsse_search.json")

# POV cam data folder with txt files
POV_CAM_FOLDER = os.path.join(INPUT_FOLDER, "readable_timelines")

# Crawled data from Acts and Pages of Homestuck
ACTS_AND_PAGES_PART_SUMMARIES = os.path.join(INPUT_FOLDER, 'part_summaries.json')
ACTS_AND_PAGES_ACT_SUMMARIES = os.path.join(INPUT_FOLDER, 'act_summaries.json')
ACTS_AND_PAGES_PAGE_SUMMARIES = os.path.join(INPUT_FOLDER, 'page_summaries.json')

# Homestuck Collection's asset pack data folder
COLLECTION_DATA_FOLDER = os.path.join(ASSET_PACK_FOLDER, "archive/data")

# Holds all the text in MS Paint Adventures, including Homestuck
MSPA_TEXT_JSON = os.path.join(COLLECTION_DATA_FOLDER, "mspa.json")
# Holds the text for news posts
NEWS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "news.json")
# Holds the text for social media posts
SOCIAL_JSON = os.path.join(COLLECTION_DATA_FOLDER, "social.json")
# Most of this is irrelevant, but holds images for additional Hussie comics
# such as Team Special Olympics
ADDITIONAL_COMICS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "comics.json")
# Holds panels
PANELS_FOLDER = os.path.join(ASSET_PACK_FOLDER, "storyfiles")
HS_PANELS_FOLDER = os.path.join(PANELS_FOLDER, "hs2")

print("Loaded constants successfully")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

print(f"OpenAI client loaded with model {MODEL_ID}")

Loading variables from .env file
...
Loaded variables successfully
Loading constants
...
Loaded constants successfully
OpenAI client loaded with model gpt-4o


# Extract MSPA data from Asset Pack

First of all, we want to open the MSPA_TEXT_JSON and fetch all the text data from it. This will be the main source of text data for our dataset.

In [4]:
"""
MSPA_TEXT_JSON format:
{
  "story": {  # The text in MS Paint Adventures, the comics themselves
    "000006": {
      "title": "Look for keyhole",
      "pageId": "000006",
      "timestamp": "1180921880",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure04.gif",
        "/advimgs/jb/mspaintadventure04b.gif"
      ],
      "content": "",
      "next": [
        "000008"
      ],
      "previous": "000005",
      "theme": "retro"
    },
    "000009": {
      "title": "Loudly tell that guy to pick up key and try it on the door.",
      "pageId": "000009",
      "timestamp": "1180931172",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure06.gif"
      ],
      "content": "Despite your bellowing, the man casually opens the door and leaves.",
      "next": [
        "000010"
      ],
      "previous": "000008",
      "theme": "retro"
    },
  },...
  "ryanquest": {...},  # Additional Ryanquest comic
  
  "faqs": {
    "general": {
      "title": "General FAQ - MS Paint Adventures",
      "pageId": "general",
      "content": "..." # html
    },
    "new": {...},  # New reader guide
    "science": {...},  # Science FAQ
    "sales": {...}, # This one was probably not Hussie, so ignore
  },
  "psExtras": {...}, # Bonus pages for Problem Sleuth with multiple images, same format with "title", "pageId" and "content" as the subkeys in "faqs"
  "wv": {...}, # "Exile" Homestuck pages with multiple images, same format as psExtras
  # Other keys are fully irrelevant
"""


def get_regular_story_records(mspa_data: dict, section_name: str):
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': page_content.get('timestamp', ''),
            'media': page_content.get('media', []),
            'next': page_content.get('next', []),
            'previous': page_content.get('previous', ''),
            'theme': page_content.get('theme', ''),
            'flag': page_content.get('flag', []),
        }
        records.append(record)
    return records


def get_media_from_html_content(content: str):
    # Extract media from HTML content
    media = []
    # Extract media URLs from the content
    matches = re.findall(r'src="([^"]+)"', content)
    for match in matches:
        # Remove the string "http://www.mspaintadventures.com" and "https://www.mspaintadventures.com" from the beginning if it exists
        match_cleaned = re.sub(r'^https?://www\.mspaintadventures\.com', '', match)
        media.append(match_cleaned)
    return media


def get_onepage_story_records(mspa_data: dict, section_name: str):
    # We need this one for faqs/psExtras/wv, which keep all information in one page and need the panels manually parsed
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': None,
            'media': get_media_from_html_content(page_content.get('content', '')),
            'next': None,
            'previous': None,
            'theme': None,
            'flag': None,
        }
        records.append(record)
    return records


# Function to load and process MSPA text data from mspa.json
def load_mspa_text_data(mspa_json_path):
    """
    Loads and processes MSPA text data from the given JSON file.

    Args:
        mspa_json_path (str): Path to the mspa.json file.

    Returns:
        pd.DataFrame: A DataFrame containing the MSPA text data.
    """
    import json
    import pandas as pd

    # Load the JSON data
    with open(mspa_json_path, 'r', encoding='utf-8') as f:
        mspa_data = json.load(f)

    # Initialize an empty list to store records
    records = []

    # Print unique sections
    print(f"Unique sections in mspa.json: {list(mspa_data.keys())}")

    # Sections to process (basically we just skip FAQs)
    records += get_regular_story_records(mspa_data, 'story')
    records += get_regular_story_records(mspa_data, 'ryanquest')
    records += get_onepage_story_records(mspa_data, 'wv')
    records += get_onepage_story_records(mspa_data, 'psExtras')

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    return df


# Now call the function and store the result in a DataFrame
mspa_df = load_mspa_text_data(MSPA_TEXT_JSON)
mspa_df

Unique sections in mspa.json: ['story', 'ryanquest', 'psExtras', 'wv', 'faqs', 'damara', 'oilRetcon']


,section,pageId,title,content,timestamp,media,next,previous,theme,flag
0,story,000006,Look for keyhole,,1180921880,"[/advimgs/jb/mspaintadventure04.gif, /advimgs/...",[000008],000005,retro,[]
1,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,[/advimgs/jb/mspaintadventure06.gif],[000010],000008,retro,[]
2,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,"[/advimgs/jb/mspaintadventure07.gif, /advimgs/...",[000011],000009,retro,[]
3,story,000011,Open door,Wonders where the fuck that pumpkin went???,1180931414,[/advimgs/jb/mspaintadventure08.gif],[000012],000010,retro,[]
4,story,000005,Grow two muscular arms with which the manhandl...,"He already has arms, stupid!",1180921870,[/advimgs/jb/mspaintadventure03.gif],[000006],000004,retro,[]
...,...,...,...,...,...,...,...,...,...,...
10049,psExtras,ps000037,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000037_1.gif, /extras/ps000037_2.gi...",None,None,None,None
10050,psExtras,ps000039,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000039_1.gif, /extras/ps000039_2.gif]",None,None,None,None
10051,psExtras,ps000034,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000034_1.gif, /extras/ps000034_2.gi...",None,None,None,None
10052,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000031_1.gif, /extras/ps000031_2.gi...",None,None,None,None


In [5]:
# Show pageId == anunsealedrecord, a special case
mspa_df[mspa_df['pageId'] == 'anunsealedtunnel']

,section,pageId,title,content,timestamp,media,next,previous,theme,flag
10006,wv,anunsealedtunnel,MS Paint Adventures,"\n\n <tbody><tr>\n <td bgcolor=""#FFFFFF""...",None,[/storyfiles/hs2/waywardvagabond/anunsealedtun...,None,None,None,None


# Creating vizPageId from pageId

The Homestuck Collection uses the MSPA page IDs, but the Homestuck website uses its own page IDs. We can convert the MSPA page IDs to the Homestuck website page IDs with some clever and totally not bruteforced heuristics. We will need to do this sooner or later because multiple upcoming datasets use the Viz page ids, which are missing in mspa_df right now.

In [6]:
# Define the adventure number to story name mapping
adventure_mapping = {
    0: "Other Section",
    1: "Jailbreak",
    2: "Bard Quest",
    3: "Blood Spade",
    4: "Problem Sleuth",
    5: "Homestuck BETA",
    6: "Homestuck",
    "ryanquest": "Ryanquest",
    "psExtras": "Problem Sleuth Extras"
}

# Define the story name to URL path mapping
story_url_mapping = {
    "Jailbreak": "jailbreak",
    "Bard Quest": "bard-quest",
    "Blood Spade": "blood-spade",
    "Problem Sleuth": "problem-sleuth",
    "Homestuck BETA": "beta",
    "Homestuck": "story",
    "Ryanquest": "ryanquest",
    "Other Section": "story/waywardvagabond"
}


def get_story_num(page_id):
    """
    Determines the story ID based on the page number.
    
    Args:
        page_id (str): The pageId, can be numeric or special identifier.
    
    Returns:
        int or str: The story number (1-6) or 'Unknown'
    """
    if page_id.isdigit():
        page_num = int(page_id.lstrip('0'))
        if page_num <= 135:
            return 1  # Jailbreak
        elif 136 <= page_num <= 216:
            return 2  # Bard Quest
        elif 219 <= page_num <= 1892:
            return 4  # Problem Sleuth
        elif 1893 <= page_num <= 1900:
            return 5  # Homestuck BETA
        elif 1901 <= page_num <= 10030:
            return 6  # Homestuck
        else:
            return "Unknown"
    else:
        # Handle special cases
        if page_id == 'mc0001':
            return 3  # Blood Spade
        elif page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
            return 6  # Homestuck
        else:
            return "Unknown"


def mspa_to_viz(row):
    """
    Converts a row's pageId and section to vizStory, vizPageNumber, vizPageId.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        pd.Series: A series with 'vizStory', 'vizPageNumber', and 'vizPageId'.
    """
    section = row['section'].lower()
    page_id = row['pageId']

    if section == 'ryanquest':
        viz_story = "Ryanquest"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"ryanquest/{viz_page_number}" \
            if isinstance(viz_page_number, int) else f"ryanquest/{viz_page_number}"
    elif section == 'story':
        story_num = get_story_num(page_id)
        if isinstance(story_num, int) and story_num in adventure_mapping:
            viz_story = adventure_mapping[story_num]
            # Adjust vizPageNumber based on story_num
            try:
                if story_num == 1:  # Jailbreak
                    viz_page_number = int(page_id.lstrip('0'))
                elif story_num == 2:  # Bard Quest
                    viz_page_number = int(page_id.lstrip('0')) - 135
                elif story_num == 3:  # Blood Spade
                    viz_page_number = "1"  # Only 'mc0001'
                elif story_num == 4:  # Problem Sleuth
                    viz_page_number = int(page_id.lstrip('0')) - 218
                elif story_num == 5:  # Homestuck BETA
                    viz_page_number = int(page_id.lstrip('0')) - 1892
                elif story_num == 6:  # Homestuck
                    if page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
                        viz_page_number = page_id  # Preserve special strings
                    else:
                        viz_page_number = int(page_id.lstrip('0')) - 1900
                else:
                    viz_page_number = None
            except:
                viz_page_number = None

            # Construct vizPageId based on vizPageNumber
            if isinstance(viz_page_number, int):
                viz_page_id = f"{story_num}/{viz_page_number}"
            elif isinstance(viz_page_number, str):
                viz_page_id = f"{story_num}/{viz_page_number}"
            else:
                viz_page_id = f"{story_num}/000000"
        else:
            viz_story = "Unknown Adventure"
            viz_page_number = None
            viz_page_id = "Unknown/000000"
    else:
        viz_story = "Other Section"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"other/{viz_page_number}" if isinstance(viz_page_number, int) else f"other/{viz_page_number}"

    return pd.Series({
        'vizStory': viz_story,
        'vizPageNumber': viz_page_number,
        'vizPageId': viz_page_id
    })


def create_viz_link(row):
    """
    Creates a visualization link based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed URL or None if information is missing.
    """
    viz_story = row['vizStory']
    viz_page_number = row['vizPageNumber']

    if pd.isnull(viz_story) or pd.isnull(viz_page_number):
        return None

    base_url = "https://homestuck.com/"
    story_path = story_url_mapping.get(viz_story, "other")

    # Handle potential edge cases where viz_page_number is not an integer
    if isinstance(viz_page_number, float):
        if viz_page_number.is_integer():
            viz_page_number = int(viz_page_number)
        else:
            # If it's not an integer, handle accordingly (e.g., set to None or round)
            return None

    return f"{base_url}{story_path}/{viz_page_number}"


mspa_df['pageId'] = mspa_df['pageId'].astype(str)

# Apply the mspa_to_viz function to create 'vizStory', 'vizPageNumber', and 'vizPageId' columns
mspa_df[['vizStory', 'vizPageNumber', 'vizPageId']] = mspa_df.apply(mspa_to_viz, axis=1)

# Create the 'vizLink' column
mspa_df['vizLink'] = mspa_df.apply(create_viz_link, axis=1)


# Convert 'vizPageNumber' to integer where possible, else keep as string for special cases
def convert_viz_page_number(x):
    if isinstance(x, int):
        return x
    elif isinstance(x, str):
        return x  # Preserve special strings
    else:
        return pd.NA


mspa_df['vizPageNumber'] = mspa_df['vizPageNumber'].apply(convert_viz_page_number)

# Display the updated DataFrame sample
mspa_df[['section', 'title', 'pageId', 'content', 'vizStory', 'vizPageNumber', 'vizPageId', 'vizLink']].sample(10)

,section,title,pageId,content,vizStory,vizPageNumber,vizPageId,vizLink
9566,story,[A6A6I5] ====>,009604,,Homestuck,7704,6/7704,https://homestuck.com/story/7704
744,story,PS: Refuse His Majesty and flip him the bird.,000782,Never! <br /><br />You gladly accept your duty...,Problem Sleuth,564,4/564,https://homestuck.com/problem-sleuth/564
2137,story,Rose: Prototype sprite with Betty Crocker box.,002175,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",Homestuck,275,6/275,https://homestuck.com/story/275
6877,story,Jake: Monstrositify something.,006918,You drop some DELICIOUS FRUIT and activate the...,Homestuck,5018,6/5018,https://homestuck.com/story/5018
1428,story,Next.,001467,The cathedral and clock tower have been sliced...,Problem Sleuth,1249,4/1249,https://homestuck.com/problem-sleuth/1249
7315,story,[A6I4] ==>,007356,The wasteland is empty. Banality skims the voi...,Homestuck,5456,6/5456,https://homestuck.com/story/5456
5311,story,==>,005354,,Homestuck,3454,6/3454,https://homestuck.com/story/3454
703,story,Next.,000741,,Problem Sleuth,523,4/523,https://homestuck.com/problem-sleuth/523
4522,story,==>,004558,,Homestuck,2658,6/2658,https://homestuck.com/story/2658
5616,story,==>,005657,"|JOURNALOG|<br /><span style=""color: #005682"">...",Homestuck,3757,6/3757,https://homestuck.com/story/3757


# Unpacking the page-based dataset into a panel-based dataset

The previous works fine for data sourced on pages instead of panels, like commentaries, but we're merging with ReadMSPA next, so we will need to `explode` the data and do some more edits, since the panel srcs will need some editing to match the transcripts.

In [7]:
mspa_panels_df = mspa_df.explode('media').reset_index(drop=True)
# Rename 'media' to 'src' for clarity
mspa_panels_df.rename(columns={'media': 'src'}, inplace=True)
# Get a sample of Homestuck panels
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6


In [8]:
# Some panel transcripts will have an id format like `00001.gif.transcript`, which in our current src is `/storyfiles/hs2/00001.gif`. We need to make a transcriptSrc to match the panel src. This applies no matter what the comics is, so we can't assume it will be /storyfiles/hs2/... we just pick the actual image file name and add '.transcript' to it.
def create_transcript_src(src):
    if pd.isnull(src):
        return None

    # Extract the image file name from the src
    image_file = src.split('/')[-1]
    # Append '.transcript' to the image file name
    transcript_src = f"{image_file}.transcript"

    return transcript_src


# Apply the function to create 'transcriptSrc' column
mspa_panels_df['transcriptSrc'] = mspa_panels_df['src'].apply(create_transcript_src)
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript


In [9]:
# Now we need to do something similar for MSPA URLs to match the key format for pages of the READMSPA format. We have a pageId like 001901, but we need to get the story_num using the matching we already have, and prepend that to the page number. So 001901 would become '6/001901' because that page number matches Homestuck. This can be the new column "pageNumber" because it's the vizPageNumber in MSPA format..
def create_page_number(row):
    """
    Creates a page number based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed page number or None if information is missing.
    """

    viz_page_number = row['pageId']

    if pd.isnull(viz_page_number):
        return None

    viz_story_number = get_story_num(str(viz_page_number))

    return f"{viz_story_number}/{viz_page_number}"


# Apply the function to create 'pageNumber' column
mspa_panels_df['pageNumber'] = mspa_panels_df.apply(create_page_number, axis=1)

# Display the updated DataFrame sample
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc,pageNumber
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript,6/001902
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript,6/001901
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript,6/001904
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript,6/001903
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript,6/001908
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript,6/001905
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript,6/001909
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript,6/001907
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript,6/001907
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript,6/001906


# Getting panel transcripts from ReadMSPA plugin

ReadMSPA's data (and its plugin from the collection) comes with text transcripts of every image's text, if not descriptions. We can use that.

In [10]:
"""
The ReadMSPA data is divided in three similar files for unknown reasons, READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE. The latter tends to have way longer transcriptions, but there are long transcriptions in READMSPA_ALTTEXT, and they have different formats besides. READMSPA_TRANSCRIPTS even contains transcripts of youtube links linked in the pages, which we should preserve anyway. 
Note the keys are not equivalent to pageId. These keys use the mspaintadventures.com format, where page numbers were padded with zeroes, as well as sharing a count for every adventure in the site, from Jailbreak to Homestuck. Homestuck's first page is 001901 and it's the adventure number 6 (instead of 4 for historical reasons, but this is not really relevant to us right now).

Portion of READMSPA_TRANSCRIPTS (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
{
    "6/001901": [
        {
            "src": null,
            "alt": "SBURB BETA",
            "contents": "SBURB BETA",
            "id": "00001.gif.transcript",
            "data": null,
            "class": [
                "transcript"
            ],
            "style": "background: white; color: #C1C1C1",
            "tag": "div"
        }
    ],
    "6/010030": [
        {
            "src": "http/www.youtube.com-rmzu89jy2j8.mp4",
            "alt": "* (This transcribes both the snapchat pics from the right of the video - which as narrative, are included in readmspa.org word count stats - and the credits from the left, which are not.)\n\n\nmspaofficial\n\nAdded Me\nAdd Friends\nMy Friends\n\n\n* HOMESTUCK\n\n* BY\n* ANDREW HUSSIE\n\n\nGreetings From\nEARTH C\n\n\n* SOUND\n* CONTRIBUTORS (A-Z)\n\n* Alex Rosetti\n* Andrew Huo\n* BurnedKirby\n* Charlie Clouser\n* Clark Powell\n* Curt Blakeslee\n* David DeCou\n* David Ko\n\n\nMAYOR\n\n\n* Dianne Warren\n* DJ Sai Tae\n* Eddie Morton\n* Eligecos\n* Erik Scheele\n* Eston Schweickart\n* First Turn Fold\n* Frank Haught\n* Gabe Nezovic\n\n\n* Insane Clown Posse\n* James Roach\n* Jan Van Den Hemel\n* Joseph Aylsworth\n* Kalibration\n* Kera Jones\n* Kevin Regamey\n* Kezinox\n* Malcolm Brown\n\n\n* Mark Hadley\n* Michael Guy Bowman\n* Michael Vallejo\n* Nick Smalley\n* Noel Sadwin\n* Paul Tuttle Starr\n* Perry Sullivan\n* Plumegeist\n* Robert J! Lake\n\n\n* Samm 413\ntentacleTherapist\ngallowsCallibrator\ntipsyGnostalgic\ngrimAuxiliatrix\ngolgothasTerror\ntimaeusTestified\ngutsyGumshoe\n\n\nCALIBORN: COME AT ME BRO.\n\n\nJOHN: step off.\n\n\nCALIBORN: COME TO MY DARK CARNIVAL. \"MOTHER FUCKER\".\n\n\nJOHN: i'll do it...\n\n\nCALIBORN: MAKE A MOVE. AND THE BUNNY GETS IT.",
            "contents": "",
            "id": "08123.mp4",
            "data": "End Credits",
            "class": [
                "flash",
                "alt-narrative"
            ],
            "style": null,
            "tag": "embed"
        }
    ],
```

Portion of READMSPA_ALTTEXT (a key matches the src column):
```json
{
    "advimgs/jb/mspaintadventure06.gif": "\nBELLOW!\n\nSLAM\n",
    "advimgs/jb/mspaintadventure09d.gif": "CRACK",
    "advimgs/jb/mspaintadventure09e.gif": "SLAM",
    "advimgs/jb/mspaintadventure05.gif": "CLANK",
    "advimgs/jb/mspaintadventure08.gif": "? ? ? ?",
    ...
    "storyfiles/hs2/07655.gif": "\nDAVE: ...\nDIRK: ...\n",
    "storyfiles/hs2/07651.gif": "nuzzzzzzzzzle",
    "storyfiles/hs2/07650.gif": "nuzzzzle",
    "storyfiles/hs2/07648.gif": "KANAYA: Shoutpole",
}

Portion of READMSPA_ALTNARRATIVE (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
"6/002153": [
        {
            "src": null,
            "alt": "<style>.page[id=\"6/002153\"] .walkaround p { border-color: #23CE27; border-radius: 0;\n\t\t       \t    \t\t    background-color: white; max-width: 100%; }</style>\n* Derived in part from <a href=\"http://pastebin.com/a3k6RgMR\">a transcript by ShadowOfFate</a>\n\n<div style=\"padding: 1em; background-color: white; border: solid 3px black\">CLICK THIS</div><dl>\n<dt>* click it!</dt>\n<dd>\n<p>To walk around, use the mouse, arrow keys, or WASD keys. Click on various objects to open command menus for them!\n\nOutstanding Flash programming by Alexis 'Gankro' Beingessner.</p>\n</dd>\n\n<dt>* click jestersprite</dt>\n<dd>\n<kbd>&gt; WHAT'S THAT\n</kbd>\n<p>It looks different now.\n\nAfter you bit that APPLE, your whole house seemed to be trasported somewhere. Then the APPLE disappeared and the KERNELSPRITE underwent a transformation. Aside from the change in appearance, the transformation doesn't seem to have any relevant ramifications. You still can't understand a word this idiot says.</p>\n<kbd>&gt; THE GHOST CLOWN...",
            "id": "00253.swf.transcript",
            "data": "John explores his house with WV (transcript and walkthrough)"
        }
    ],
    "6/002149": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://mspaintadventures.wikia.com/wiki/Rose%27s_Walkthrough\">MSPA Wiki</a>\n\n<p style=\"background-color: black; color: white; padding: 1em;\n          font-family: Lucida Console, courier;\"><b>Sburb Beta FAQ/Walkthrough</b> by <span style=\"color: #ff6600\">tentacleTherapist</span>\n<b>Hosted by</b> <span style=\"color: #ff6600\">GameFAQs</span>\nVersion 1.0, Last Updated 2009-04-13\n<span style=\"color: #ff6600\">View/Download Original File</span>\nLiked this FAQ? Click to <span style=\"color: #ff6600\">recommend</span> it to other GameFAQs users.\n</p>\n<article style=\"font-family: Lucids Console, courier\">\nSburb Beta Walkthrough\nVersion 1.0, April 13, 2009\nBy tentacleTherapist\n\n=============================== TABLE OF CONTENTS ===============================\n\n1. Caveats and Condolences........................... [0000]\n2. Walkthrough (Incomplete).......................... [A000]\n2.1. An Examination of the Basics.................... [A000]\n2.2. So your cruxtruder is ticking. Do this to live.. [A100]\n2.3. The Long and Short. The Medium too.............. [B100]\nsome stuff about captcha codes and punch card alchemy [Z001]\nC. Appendix 3 -- Screen Captures, pt. 1.............. [Z301]\n?. Rose: Egress...",
            "id": "00249_2.gif.transcript",
            "data": "Rose's Sburb Beta Walkthrough - Caveats and Condolences"
        }
    ],
    "6/002037": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://pastebin.com/FQhpyeKx\">a transcript by ShadowOfFate</a>\n\n<ul style=\"list-style: none; background-color: #4CE24E; padding: 1em;\n    color: white; text-align: center; white-space: normal\">\n<li>Transforming Soffits</li>\n<li>Reorganizing Keys</li>\n<li>Formalizing Immersion Joints</li>\n<li>Justifying Kick Extractors</li>\n<li>Advising Aggregates</li>\n<li>Managing Elbows</li>...",
            "id": "00137.swf.transcript",
            "data": "Sburb Beta loading screen"
        }
    ]
}

Our objective here is to combine the information so that, for each panel, we'll have its text transcript.
"""


# Extract the ReadMSPA data from the JSON files
def load_readmspa_data(transcripts_path, alttext_path, altnarrative_path):
    """
    Loads and processes the ReadMSPA data from the given JSON files.

    Args:
        transcripts_path (str): Path to the transcripts JSON file.
        alttext_path (str): Path to the alttext JSON file.
        altnarrative_path (str): Path to the altnarrative JSON file.

    Returns:
        pd.DataFrame: A DataFrame containing the ReadMSPA data.
    """
    # Load the JSON data
    with open(transcripts_path, 'r', encoding='utf-8') as f:
        transcripts_data = json.load(f)
    with open(alttext_path, 'r', encoding='utf-8') as f:
        alttext_data = json.load(f)
    with open(altnarrative_path, 'r', encoding='utf-8') as f:
        altnarrative_data = json.load(f)

    # Initialize an empty list to store page records
    page_records = []

    # Iterate over each data dictionary of READMSPA_TRANSCRIPTS and READMSPA_ALTNARRATIVE
    for data_dict in [transcripts_data, altnarrative_data]:
        # Iterate over each key-value pair
        for key, value_list in data_dict.items():
            # Extract relevant fields
            for value in value_list:
                record = {
                    'pageNumber': key,
                    'src': value.get('src', None),
                    'contents': value.get('contents', ''),
                    'transcriptSrc': value.get('id', ''),
                    'data': value.get('data', ''),
                    'class': value.get('class', []),
                    'style': value.get('style', ''),
                    'tag': value.get('tag', ''),
                    'alt': value.get('alt', ''),
                }
                page_records.append(record)

    # Initialize an empty list to store alttext records
    panel_records = []

    # Iterate over each key-value pair in READMSPA_ALTTEXT
    for key, value in alttext_data.items():
        # Extract relevant fields
        record = {
            'src': key,
            'contents': value,
        }
        panel_records.append(record)

    # Create DataFrames from the records
    page_df = pd.DataFrame(page_records)
    panel_df = pd.DataFrame(panel_records)

    return page_df, panel_df


# Now call the function and store the results in DataFrames
readmspa_pages_df, readmspa_panels_df = load_readmspa_data(READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT,
                                                           READMSPA_ALTNARRATIVE)

# Display the updated DataFrame samples
readmspa_pages_df.sample(5)


,pageNumber,src,contents,transcriptSrc,data,class,style,tag,alt
224,6/002037,None,,00137.swf.transcript,Sburb Beta loading screen,[],,,"* Derived from <a href=""http://pastebin.com/FQ..."
99,6/006727,None,"<style>.page[id=""6/006727""] .transcript p { ba...",04824.swf.transcript,"MystStuck, Part 3 - Gamzee's potions (transcri...","[transcript, alt-narrative]",,div,"<style>.page[id=""6/006727""] .transcript p { ba..."
8,6/009736,None,"<span style=""color: #0715cd; font-size: 400%;""...",07832_2.gif.transcript,None,[transcript],,div,"<span style=""color: #0715cd; font-size: 400%;""..."
197,6/005556,storyfiles/hs2/03653_2.gif,,03653_2.gif,The Yellow Yard,[],,,you propaply want to know\nhow INVOVLED\nthe b...
52,6/008412,None,"<div class=""log""><section>\n<span style=""color...",06509.gif.transcript,horsesuck,[transcript],,div,"<div class=""log""><section>\n<span style=""color..."


In [11]:
readmspa_panels_df.sample(5)

,src,contents
2519,storyfiles/hs2/scraps/wheredoingitman.gif,where doing it man
474,storyfiles/hs2/06100_retcon.gif,-OMESTUCK
180,storyfiles/hs2/06936.gif,\nZILLYLOOT!\n\nDISCOVER\n\nHONK WHEE TOOT BOI...
829,storyfiles/hs2/04971.gif,Alert : Ũ
3480,advimgs/jb/mspaintadventure20b.gif,CRACK


In [12]:
# Now we can merge readmspa_pages_df on its id (which will match transcriptSrc) with mspa_panels_df on transcriptSrc
# Note we only care about 
mspa_panels_df = mspa_panels_df.merge(readmspa_pages_df, on='transcriptSrc', how='left',
                                      suffixes=('', '_readmspapages'))
# Drop duplicate rows, we just want one record per "src".
mspa_panels_df.drop_duplicates(subset='src', inplace=True)
# Show some panels with transcripts
mspa_panels_df[~mspa_panels_df['contents'].isnull()].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcriptSrc,pageNumber,pageNumber_readmspapages,src_readmspapages,contents,data,class,style,tag,alt
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],...,00002.gif.transcript,6/001902,6/001902,None,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\..."
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],...,00001.gif.transcript,6/001901,6/001901,None,SBURB BETA,None,[transcript],background: white; color: #C1C1C1,div,SBURB BETA
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],...,00003.gif.transcript,6/001903,6/001903,None,"<b style=""color: green"">✓</b> <b style=""color:...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: green"">✓</b> <b style=""color:..."
2408,story,001912,John: Read note on drawer.,This note is rich with the aromas of FATHERLY ...,1239679025,/storyfiles/hs2/00012.gif,[001913],001911,,[],...,00012.gif.transcript,6/001912,6/001912,None,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.,None,"[transcript, dad-note]",,div,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.
2562,story,002037,[S] ==>,,1241865178,/storyfiles/hs2/00137/00137.swf,[002038],002036,,"[F, HQ]",...,00137.swf.transcript,6/002037,6/002037,None,"* Derived from <a href=""http://pastebin.com/FQ...",Sburb Beta loading screen,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://pastebin.com/FQ..."
2694,story,002149,ACT 2 ==>,,1244666782,/storyfiles/hs2/00249_2.gif,[002150],002148,,[],...,00249_2.gif.transcript,6/002149,6/002149,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - Caveats and Co...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2704,story,002153,[S] YOU THERE. BOY.,,1244949337,/storyfiles/hs2/00253/00253.swf,[002154],002152,,[F],...,00253.swf.transcript,6/002153,6/002153,None,"<style>.page[id=""6/002153""] .walkaround p { bo...",John explores his house with WV (transcript an...,"[transcript, alt-narrative, walkaround]",,div,"<style>.page[id=""6/002153""] .walkaround p { bo..."
2726,story,002172,==>,,1245374550,/storyfiles/hs2/00272_2.gif,[002173],002171,,[],...,00272_2.gif.transcript,6/002172,6/002172,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - An Examination...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2919,story,002340,==>,"She's not finished with this yet! Jeez, cut he...",1249444721,/storyfiles/hs2/00440_2.gif,[002341],002339,,[],...,00440_2.gif.transcript,6/002340,6/002340,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - The Long and S...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2998,story,002410,==>,"<a href=""http://www.mspaintadventures.com/stor...",1250822032,/storyfiles/hs2/00510_2.gif,[002411],002409,,[],...,00510_2.gif.transcript,6/002410,6/002410,None,"<article style=""font-family: Lucida Console, c...",Rose's Sburb Beta Walkthrough - Screen Captures,"[transcript, alt-narrative]",,div,"<article style=""font-family: Lucida Console, c..."


In [13]:
# Merge the panel data too, this time on readmspa_panels_df's src, which matches mspa_panels_df's src, but, critically, without the slash at the beginning. 
# Make a copy of the src column in readmspa_panels_df to match the src column in mspa_panels_df
mspa_panels_df['src_copy'] = mspa_panels_df['src'].apply(lambda x: x[1:] if x.startswith('/') else x)
# Merge the DataFrames
mspa_panels_annotated_df = mspa_panels_df.merge(readmspa_panels_df, left_on='src_copy', right_on='src', how='left',
                                                suffixes=('', '_readmspanels'))
# Show a panel with annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,src_readmspapages,contents,data,class,style,tag,alt,src_copy,src_readmspanels,contents_readmspanels
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,storyfiles/hs2/01085_2.gif,storyfiles/hs2/01085_2.gif,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [14]:
# Time to clean up, now that we know transcripts in either "alt" or "contents_readmspanels"
# Useless columns are "transcriptSrc", "pageNumber_readmspapages", "src_readmspapages", "contents", "data", "src_copy", "src_readmspanels"
# "class", "style" and "tag" should be renamed to "transcript_class", "transcript_style" and "transcript_tag" respectively
# "transcript" should be a new column that contains the value of either "alt" or "contents_readmspanels"
# "alt" and "contents_readmspanels" should be dropped afterwards

# Drop the unnecessary columns
mspa_panels_annotated_df.drop(
    columns=['transcriptSrc', 'pageNumber_readmspapages', 'src_readmspapages', 'contents', 'data', 'src_copy',
             'src_readmspanels'], inplace=True)

# Rename the 'class', 'style', and 'tag' columns
mspa_panels_annotated_df.rename(
    columns={'class': 'transcript_class', 'style': 'transcript_style', 'tag': 'transcript_tag'}, inplace=True)

# Create the 'transcript' column
mspa_panels_annotated_df['transcript'] = mspa_panels_annotated_df['alt'].fillna(
    mspa_panels_annotated_df['contents_readmspanels'])

# Drop the 'alt' and 'contents_readmspanels' columns
mspa_panels_annotated_df.drop(columns=['alt', 'contents_readmspanels'], inplace=True)

# Show a panel with cleaned annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],Homestuck,1085,6/1085,https://homestuck.com/story/1085,6/002985,NaN,NaN,NaN,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [15]:
mspa_panels_annotated_df

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
0,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
1,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04b.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
2,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,/advimgs/jb/mspaintadventure06.gif,[000010],000008,retro,[],Jailbreak,9,1/9,https://homestuck.com/jailbreak/9,1/000009,NaN,NaN,NaN,\nBELLOW!\n\nSLAM\n
3,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
4,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07b.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000031_3.gif,None,None,None,None,Other Section,ps000031,other/ps000031,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000031,NaN,NaN,NaN,NaN
11692,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_1.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11693,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_2.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11694,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_3.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN


# Getting panel tags from HSSE

The Homestuck Search Engine people tagged the tags of the first four acts, from characters to locations and more. Will be extremely useful for image transcription.

In [16]:
"""
HSSE_TAGS and HSSE_SEARCH file contain the entirety of the Homestuck Search Engine tagged data (only the first four acts, until page 1988 inclusive and excluding some swfs) in its own bespoke JSON format.
HSSE_TAGS is the simpler json with tag definitions, and which tags contain other tags:
```json
 "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
    ...
}
```

HSSE_SEARCH is the more complex json with the actual tags for each panel:
```json
[
  {
    "_id": 0,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "url": "https://homestuck.com/story/1",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      749,
      801,
      1301,
      602,
      1192,
      711,
      1349
    ],
    "page": 1
  },
  {
    "_id": 1,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "url": "https://homestuck.com/story/2",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      1349,
      602
    ],
    "page": 2
  },
  ...
]
```
Our objective here is to combine the information so that, for each page, we'll have its human readable tags. 
"""


def load_hsse_tags(tags_path):
    """
    Loads and processes the HSSE tag definitions.

    Args:
        tags_path (str): Path to the hsse_tags.json file.

    Returns:
        dict: A dictionary mapping tag IDs (int) to tag names (str).
    """
    with open(tags_path, 'r', encoding='utf-8') as f:
        tags_data = json.load(f)

    # Extract the 'definitions' key
    definitions = tags_data.get('definitions', {})

    # Build mapping from _id (int) to name (str)
    tag_id_to_name = {}
    for tag_id_str, tag_info in definitions.items():
        tag_id = tag_info.get('_id')
        name = tag_info.get('name')
        if tag_id is not None and name is not None:
            tag_id_to_name[tag_id] = name
    return tag_id_to_name


def load_hsse_search(search_path):
    """
    Loads and processes the HSSE page-tag associations.

    Args:
        search_path (str): Path to the hsse_search.json file.

    Returns:
        dict: A dictionary mapping page IDs (str) to lists of tag IDs (int).
    """
    with open(search_path, 'r', encoding='utf-8') as f:
        search_data = json.load(f)

    # search_data is a list of dicts
    page_to_tag_ids = {}
    for entry in search_data:
        page = entry.get('page')  # integer
        tags = entry.get('tags', [])  # list of integers
        if page is not None:
            page_str = str(page)  # Convert page number to string to match 'pageId'
            page_to_tag_ids[page_str] = tags

    return page_to_tag_ids


def map_tags_to_names(page_to_tag_ids, tag_id_to_name):
    """
    Maps tag IDs to tag names for each page.

    Args:
        page_to_tag_ids (dict): Dictionary mapping page IDs to lists of tag IDs.
        tag_id_to_name (dict): Dictionary mapping tag IDs to tag names.

    Returns:
        dict: Dictionary mapping page IDs to lists of tag names.
    """
    page_to_tag_names = {}
    for page_id, tag_ids in page_to_tag_ids.items():
        # Ensure tag_ids is a list
        if not isinstance(tag_ids, list):
            tag_ids = [tag_ids]
        # Map each tag ID to its name, handle unknown tags gracefully
        tag_names = [tag_id_to_name.get(tag_id, f"Unknown Tag {tag_id}") for tag_id in tag_ids]
        page_to_tag_names[page_id] = tag_names
    return page_to_tag_names


def merge_tags_into_mspa_df(mspa_df, page_to_tag_names):
    """
    Merges HSSE tag data into the main MSPA DataFrame.

    Args:
        mspa_df (pd.DataFrame): The main MSPA DataFrame with transcripts.
        page_to_tag_names (dict): Dictionary mapping page IDs to lists of tag names.

    Returns:
        pd.DataFrame: The updated MSPA DataFrame with tags incorporated.
    """
    # Edit page_to_tag_names so the keys have the "6/" prefix to match the Viz page numbers
    page_to_tag_names = {f"6/{k}": v for k, v in page_to_tag_names.items()}

    # Create a Series from the page_to_tag_names dictionary
    tags_series = pd.Series(page_to_tag_names, name='tags')

    # Merge the tags into mspa_df, matching its vizPageId with the keys of the tags_series
    mspa_df_with_tags = mspa_df.merge(tags_series, left_on='vizPageId', right_index=True, how='left')

    # Replace NaN with empty lists for pages without tags
    mspa_df_with_tags['tags'] = mspa_df_with_tags['tags'].apply(lambda x: x if isinstance(x, list) else [])

    return mspa_df_with_tags


# Load HSSE tag definitions
tag_id_to_name = load_hsse_tags(HSSE_TAGS)
print(f"Loaded {len(tag_id_to_name)} tags from hsse_tags.json.")

# Load HSSE page-tag associations
page_to_tag_ids = load_hsse_search(HSSE_SEARCH)
print(f"Loaded tag associations for {len(page_to_tag_ids)} pages from hsse_search.json.")

# Map tag IDs to tag names
page_to_tag_names = map_tags_to_names(page_to_tag_ids, tag_id_to_name)
print("Mapped tag IDs to tag names.")

# Merge tags into the main MSPA DataFrame
mspa_df_with_tags = merge_tags_into_mspa_df(mspa_panels_annotated_df, page_to_tag_names)

# # Display the first few rows with tags not an empty list
mspa_df_with_tags[mspa_df_with_tags['tags'].apply(len) > 0].sample(10)

Loaded 1427 tags from hsse_tags.json.
Loaded tag associations for 8002 pages from hsse_search.json.
Mapped tag IDs to tag names.


,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags
3322,story,002728,Jade: See if Dave left you a sweet new rap.,"It does not appear so, but you just never know...",1256890473,/storyfiles/hs2/00828.gif,[002729],002727,,[],Homestuck,828,6/828,https://homestuck.com/story/828,6/002728,NaN,NaN,NaN,\nturntechGodhead [TG] began pestering gardenG...,"[Part 1, Act 3, Screen, Pesterchum]"
3007,story,002463,Dave: Take expensive ninja sword.,What sword?,1251701530,/storyfiles/hs2/00563.gif,[002464],002462,,[],Homestuck,563,6/563,https://homestuck.com/story/563,6/002463,NaN,NaN,NaN,NaN,"[Part 1, Act 2, Dave's House, Dave Strider, Re..."
3603,story,002954,John: Combine ghost shirt and suit.,You make the GREEN SLIME GHOST SUIT.<br /><br ...,1261449453,/storyfiles/hs2/01054_2.gif,[002955],002953,,[],Homestuck,1054,6/1054,https://homestuck.com/story/1054,6/002954,NaN,NaN,NaN,NaN,"[Part 1, Act 3, John Egbert, Green Slime Ghost..."
4559,story,003717,Jade: Build.,You take advantage of Dave's nap to make some ...,1273625363,/storyfiles/hs2/01817.gif,[003718],003716,,[],Homestuck,1817,6/1817,https://homestuck.com/story/1817,6/003717,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Jadebot, Screen, Keyboard, Fro..."
4553,story,003720,Dave: Wake up and jam.,And by jam you mean alchemize of course.<br />...,1273708435,/storyfiles/hs2/01820_1.gif,[003721],003719,,[],Homestuck,1820,6/1820,https://homestuck.com/story/1820,6/003720,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dave Strider, Suit, Puppet Tux..."
4006,story,003264,==>,,1265865636,/storyfiles/hs2/01364_2.gif,[003265],003263,,[],Homestuck,1364,6/1364,https://homestuck.com/story/1364,6/003264,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Hero Mode, Carapacian, Exile, ..."
2724,story,002224,==>,|PESTERLOG|<br />-- turntechGodhead <span styl...,1246520170,/storyfiles/hs2/00324.gif,[002225],002223,,[],Homestuck,324,6/324,https://homestuck.com/story/324,6/002224,NaN,NaN,NaN,\n:: ectoBiologist ::\n\n-- turntechGodhead [T...,"[Act 2, Part 1, Screen, Hephaestus, Browser, P..."
3313,story,002721,==>,You switch your ECLECTIC BASS to its advanced ...,1256635596,/storyfiles/hs2/00821.gif,[002722],002720,,[],Homestuck,821,6/821,https://homestuck.com/story/821,6/002721,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Jade Harley, Jade's Room, Jade..."
3300,story,002706,Jade: Captchalogue refrigerator.,You take the REFRIGERATOR.<br /><br />You migh...,1256419618,/storyfiles/hs2/00806_2.gif,[002707],002705,,[],Homestuck,806,6/806,https://homestuck.com/story/806,6/002706,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Sylladex, Fetch Modus, Captcha..."
4425,story,003609,==>,,1271710216,/storyfiles/hs2/01709.gif,[003610],003608,,[],Homestuck,1709,6/1709,https://homestuck.com/story/1709,6/003609,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert, Package, Lined Mo..."


# Extracting character appearances from POV cam 

The POV cam extension for Homestuck allows us to see the characters that are present in each page, and not just until page 1988, all of them. We can use this to extract character tags and somewhat make up for the lack of tags in the later pages.

In [17]:
"""
The data from the POV cam comes in many files named after each character, like "roxy.txt" and "rufioh.txt". The format is not meant to be easily parsable, but it shouldn't be too hard to extract the data and "invert" it, to get the characters that appear in each page and their "commands".

An example of the data (jade.txt):
```
Name: Jade
Colour: #4AC925
Image: jade.png
Group: Kids

Be created on meteor
3790-3791
3803
3807
3830-3831

Be sent to Earth
3840

Land on factory
3768-3769

Be adopted
3773-3775

Be taken on hunt with grandfather
Wander off with Bec
Find present
3029-3036
```
From the documentation:
```md
## Timeline language

In the `Readable Timelines` folder are a number of files, each containing the timeline data for a single person.

The files use the following format:

 * Page numbers or ranges of numbers to describe what pages a person's on.
   (For A6A5A1x2 COMBO, use `-2` on the end to go through the pages two at a time)
   eg. `6009`, `1901-2032`, or `7688-7692-2`
 * To split the timeline, indent the splintered timeline, then return to the original indentation for the alpha timeline.
   Note that the two timelines are not connected by default, you must use the next two commands to describe how they should be joined.
 * `==>`: Jump into the next split timeline from this point
 * `<==`: Jump out of previous split timeline to this point
 * `~`: Insert another timeline here, using a person's name.
   eg. `~ Davesprite`

The following commands change properties about the current person or timeline.
Write the exact word, then `:`, then the value you wish to set it to.
eg. `Name: John`.

 * `Name`: Change the name of the current person.
 * `Colour`: Change the colour used for the links.
 * `Image`: Change the image used for the links.
 * `Group`: Change which group the links are a part of.
 * `Caption`: Give some hover-over text to the link between the previous page and the next.

All lines which do not fit any of the above are ignored, like comments.
```
"""


def extract_data_from_pov_cam_file(pov_cam_file: str) -> pd.DataFrame:
    """
    Extracts data from a POV cam file.

    Args:
        pov_cam_file (str): The path to the POV cam file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data with columns ['page', 'character', 'commands'].
    """
    # Regular expression to identify page lines (single number or range with optional step)
    page_pattern = re.compile(r'^\d+(-\d+)?(-\d+)?$')

    try:
        with open(pov_cam_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File not found - {pov_cam_file}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])
    except Exception as e:
        print(f"Error reading file {pov_cam_file}: {e}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])

    # Initialize the data storage
    data = {
        'page': [],
        'character': [],
        'commands': []
    }

    # Initialize current character and commands
    current_character = None
    current_commands = []
    last_commands = []

    for line_number, line in enumerate(lines, start=1):
        original_line = line  # Keep the original line for debugging if needed
        line = line.strip()

        if not line:
            continue  # Skip empty lines

        # Check for metadata lines
        if line.startswith("Name: "):
            # Extract character name from the 'Name:' line
            extracted_name = line.split("Name: ", 1)[1].strip()
            if current_character is None:
                current_character = extracted_name
            else:
                # Ignore subsequent 'Name:' lines to prevent name changes from affecting the character name
                continue
            current_commands = []  # Reset commands when a new character is found
            last_commands = []  # Reset last_commands for the new character
            continue  # Move to the next line
        elif any(line.startswith(prefix) for prefix in ["Colour: ", "Image: ", "Group: "]):
            continue  # Skip other metadata lines

        # Check if the line is a page line
        if page_pattern.match(line):
            if current_commands:
                # If there are new commands, update last_commands
                last_commands = current_commands.copy()
                commands_to_use = current_commands.copy()
            elif last_commands:
                # If no new commands, inherit last_commands
                commands_to_use = last_commands.copy()
            else:
                # If no commands to inherit, assign ["==>"]
                commands_to_use = ["==>"]

            # Parse the page or range
            pages = []
            parts = line.split('-')
            try:
                if len(parts) == 1:
                    pages = [int(parts[0])]
                elif len(parts) == 2:
                    start, end = map(int, parts)
                    pages = list(range(start, end + 1))
                elif len(parts) == 3:
                    start, end, step = map(int, parts)
                    pages = list(range(start, end + 1, step))
                else:
                    print(f"Warning: Unexpected page format at line {line_number}: '{original_line}'")
                    continue  # Skip unexpected formats
            except ValueError:
                print(f"Error: Non-integer page numbers at line {line_number}: '{original_line}'")
                continue  # Skip lines with non-integer values

            for page in pages:
                if page in data['page']:
                    # If the page already exists, append the new commands without duplication
                    index = data['page'].index(page)
                    existing_commands = data['commands'][index]
                    for cmd in commands_to_use:
                        if cmd not in existing_commands:
                            existing_commands.append(cmd)
                else:
                    data['page'].append(page)
                    data['character'].append(current_character)
                    data['commands'].append(commands_to_use.copy())  # Use a copy to prevent mutation

            current_commands = []  # Reset commands after assigning them to pages
        else:
            # Assume it's a command line
            current_commands.append(line)

    # Remove duplicate commands per page while preserving order
    for i, cmds in enumerate(data['commands']):
        seen = set()
        unique_cmds = []
        for cmd in cmds:
            if cmd not in seen:
                unique_cmds.append(cmd)
                seen.add(cmd)
        data['commands'][i] = unique_cmds

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


def extract_pov_cam_data(pov_cam_folder: str) -> pd.DataFrame:
    """
    Extracts data from all POV cam files in a given folder.

    Args:
        pov_cam_folder (str): The path to the folder containing POV cam text files.

    Returns:
        pd.DataFrame: A combined DataFrame containing data from all POV cam files.
    """
    # List all .txt files in the folder
    pov_cam_files = [file for file in os.listdir(pov_cam_folder) if file.endswith('.txt')]
    pov_cam_file_with_full_path = [os.path.join(pov_cam_folder, file) for file in pov_cam_files]

    # Initialize a list to hold DataFrames from each file
    pov_cam_data_frames = []

    for file_path in pov_cam_file_with_full_path:
        df = extract_data_from_pov_cam_file(file_path)
        if not df.empty:
            pov_cam_data_frames.append(df)
        else:
            print(f"No data extracted from {file_path}")

    if pov_cam_data_frames:
        pov_cam_data = pd.concat(pov_cam_data_frames, ignore_index=True)
    else:
        pov_cam_data = pd.DataFrame(columns=['page', 'character', 'commands'])

    return pov_cam_data


# Extract data from the POV cam files
pov_cam_data = extract_pov_cam_data(POV_CAM_FOLDER)

# Display the extracted data
pov_cam_data.sample(10)

,page,character,commands
17727,9761,Vriska,[Tavros.]
17143,8937,Terezi,"[3V3RYBODY 1S D34D, JOHN, Ask John to think up..."
13136,9864,Quarters,[Be dropped off on LOMAX by SS]
13722,5682,Rose,"[Round Two., Be killed]"
11993,5476,Lil' Cal,"[Be thrown at Aradiabot, Be sent back in time,..."
4333,9984,Eggs,[Be dropped off on LOMAX by SS]
1495,8763,Caliborn,"[Continue Homosuck, Be interrupted by John, GA..."
15365,6276,Sollux,[Meet friends]
14354,6522,Roxy,[Be snuck up upon by Dersites]
296,5505,Aradia,[======>]


In [18]:
# Show some unique complex data, like pages with multiple commands
pov_cam_data[pov_cam_data['commands'].apply(len) > 2].sample(10)

,page,character,commands
710,8459,Aranea,"[Mind control Gamzee to get ring, Become alive..."
15534,3166,Spades Slick,"[Remove Crowbar's pin., Snowman's introduction..."
10511,9448,John,"[Compliment Vriska's strategic analysis, Be as..."
14042,7300,Rose,"[Year 2, Prepare for date with Kanaya, Be drunk]"
5028,9230,Gamzee,"[HONK, O_O, Notice John, Be noticed by Terezi]"
3538,6441,Dirk,"[Be introduced, Rap off with Squarewave, Be sn..."
1491,8759,Caliborn,"[Continue Homosuck, Be interrupted by John, GA..."
9672,7240,John,"[Enter dream bubble, Fight Jack Noir, Learn ne..."
4319,8034,Eggs,"[Be resurrected by SS, Be commanded by SS, Lea..."
7788,6346,Jane,"[Do some sorta lifey thing, Wake up, Talk with..."


In [19]:
# Now that we have the page/character/commands dataset, we can merge the page/character data
# We'll turn the commands into an array of commands prefixed with the character name
# We'll also add a "pov_characters" and "pov_commands" column to the mspa_df
# Note that for the POV cam, the vizPageId is actually incorrect. We can use the actual page number in pageId instead,
# though it's six padded digits in mspa_df_with_tags and an actual integer in pov_cam_data. We'll padd the latter to match.

# Pad the page numbers in pov_cam_data
pov_cam_data['pageId'] = pov_cam_data['page'].apply(lambda x: f"{x:06d}")

# Merging the data now would be silly, we need to create a new dataframe where there's only one row per page
# We'll concatenate the commands for each character in the same page
# We'll also add the commands to a list, and the characters to a list

# Add the character names to the commands if the commands don't already have them
pov_cam_data['commands'] = pov_cam_data.apply(lambda x: [f"{x['character']}: {cmd}" for cmd in x['commands']], axis=1)

# Group by pageId and character
grouped = pov_cam_data.groupby(['pageId']).agg({
    'character': list,
    'commands': list
}).reset_index()

grouped.sample(10)

,pageId,character,commands
6716,008930,"[John, Roxy, Terezi]","[[John: Notice Jade's shoes, John: Be told of ..."
2644,004620,[Kanaya],[[Kanaya: Hop to 8=8.]]
4456,006528,[Roxy],[[Roxy: Run for fenestrated plane]]
4052,006107,[Jane],"[[Jane: Sneak out of house, Jane: Talk with Ja..."
2715,004692,"[Aradia, Equius, Eridan, Feferi, Gamzee, Kanay...","[[Aradia: Arrive on meteor], [Equius: Explain ..."
785,002692,[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]"
6843,009059,"[John, Terezi]","[[John: ???????, John: <==, John: there's no p..."
763,002670,[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]"
7346,009572,"[Dave, Dirk]","[[Dave: Character select], [Dirk: Character se..."
3843,005882,"[Jade, Kanaya, Karkat]","[[Jade: Talk with Karkat, Kanaya, Jade: Rememb..."


In [20]:
# Merge the data into mspa_df_with_tags
# "character" can become "pov_characters" and "commands" can become "pov_commands"

# rename columns
grouped.rename(columns={'character': 'pov_characters', 'commands': 'pov_commands'}, inplace=True)

mspa_df_with_pov = mspa_df_with_tags.merge(grouped, on='pageId', how='left')

# Replace NaN with empty lists for pages without POV cam data
mspa_df_with_pov['pov_characters'] = mspa_df_with_pov['pov_characters'].apply(
    lambda x: x if isinstance(x, list) else [])
mspa_df_with_pov['pov_commands'] = mspa_df_with_pov['pov_commands'].apply(lambda x: x if isinstance(x, list) else [])

mspa_df_with_pov[mspa_df_with_pov['pov_characters'].apply(len) > 0].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
7467,story,006222,[A6I1] ==>,,1323830557,/storyfiles/hs2/04319.gif,[006223],006221,,[],...,6/4322,https://homestuck.com/story/4322,6/006222,NaN,NaN,NaN,NaN,[],"[Parcel Mistress, Warweary Villein]","[[Parcel Mistress: Take ring, Parcel Mistress:..."
10988,story,009529,[A6A6I5] ====>,,1433286024,/storyfiles/hs2/07625.gif,[009530],009528,,[],...,6/7629,https://homestuck.com/story/7629,6/009529,NaN,NaN,NaN,NaN,[],"[Calliope, Roxy]","[[Calliope: Be surprised by Roxy, Calliope: Re..."
10349,story,008899,[A6A6I4] ====>,,1415597504,/storyfiles/hs2/06996.gif,[008900],008898,,[],...,6/6999,https://homestuck.com/story/6999,6/008899,NaN,NaN,NaN,NaN,[],"[John, Roxy, Terezi]","[[John: Fly around, John: Find Dirk, John: Fol..."
11458,story,009982,[A6A6I5] ====>,,1459620887,/storyfiles/hs2/08075_2.gif,[009983],009981,,[],...,6/8082,https://homestuck.com/story/8082,6/009982,NaN,NaN,NaN,NaN,[],[Caliborn],[[Caliborn: Visit Yaldabaoth]]
9584,story,008165,CONFIRM THE KILL.,YOU (I) PRODUCE PHOTOGRAPHIC EVIDENCE OF THE S...,1371080587,/storyfiles/hs2/06263.gif,[008166],008164,A6A6,[],...,6/6265,https://homestuck.com/story/6265,6/008165,NaN,NaN,NaN,\nKOD.\n\nDEAD.\n,[],[Caliborn],"[[Caliborn: Destroy discs 1, 2, Caliborn: Info..."
8800,story,007487,==>,"|PESTERLOG|<br /><span style=""color: #ff6ff2"">...",1355551207,/storyfiles/hs2/05584.gif,[007488],007486,,[],...,6/5587,https://homestuck.com/story/5587,6/007487,NaN,NaN,NaN,NaN,[],[Roxy],"[[Roxy: In dream bubble, Roxy: Doint write whi..."
8386,story,007087,Be Dirk.,"|PESTERLOG|<br /><span style=""color: #323232"">...",1340701139,/storyfiles/hs2/05184.gif,[007088],,,[],...,6/5187,https://homestuck.com/story/5187,6/007087,NaN,NaN,NaN,NaN,[],"[Caliborn, Dirk]","[[Caliborn: <==, Caliborn: Jeer Calliope about..."
5938,story,004894,==>,"|PESTERLOG|<br /><span style=""color: #77003c"">...",1291537369,/storyfiles/hs2/02993_retcon.gif,[004895],004893,,[R4],...,6/2994,https://homestuck.com/story/2994,6/004894,NaN,NaN,NaN,NaN,[],"[Feferi, Jade]","[[Feferi: Meet Jade in dream bubble], [Jade: M..."
3286,story,002700,Jade: Change wardrobifier setting.,You deactivate the WARDROBIFIER'S randomizatio...,1256257688,/storyfiles/hs2/00800_1.gif,[002701],002699,,[],...,6/800,https://homestuck.com/story/800,6/002700,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Wardrobifier, Light Symbol, Be...",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]"
4160,story,003383,==>,You follow the agent. You must not lose track ...,1267052482,/storyfiles/hs2/01483.gif,[003384],003382,,[],...,6/1483,https://homestuck.com/story/1483,6/003383,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Exile, Carapacian, PM, Dersite...","[Authority Regulator, Parcel Mistress]","[[Authority Regulator: Ticket car], [Parcel Mi..."


# Adding Acts and Pages of Homestuck data

I completely forgot this existed until I was already trying to finetune a model, but this is critical information. For each act and even groups of pages, we have detailed synopses. We'll actually need to crawl and generate the dataset ourselves.

In [21]:
"""
The target is https://rafe.name/homestuck.
We'll obviously have to crawl all the act pages, and the part pages. Each part contains multiple acts, each act contains multiple groups of pages. 
We'll end up with a dataset like...
page_id | page_group_summary | act | act_summary | part | part_summary
1 | "John Egbert examines his room." | "Act 1" | [["JOHN", "Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert...."], ["SBURB", "The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester..."]] | "Part 1" | "John, Rose, Dave, and Jade are online friends. On John’s thirteenth birthday, the kids play a game that takes them and their guardians to a set of fantastic worlds called the Medium. They are challenged to work together, kill monsters, learn punch-card alchemy, and build their houses up to the heavenly orb of Skaia..."

This should be really useful when we're trying to guess the tags that are in a page. The summaries should also be generally useful for text-based models later on. 
(Note: obviously I ended up making three datasets instead because the act/part transcripts are HUUUUUUUUUGE.)
"""

ACTS_AND_PAGES_URL = "https://rafe.name/homestuck/"

# Once we have the file in the repo, no need to crawl again
FORCE_ACTS_CRAWL = False

# Rafe never made a summary of part 4, so I made one myself
PART_FOUR_SUMMARY = """Caliborn seizes control of the narrative. He replaces Homestuck with his own twisted version, "Homosuck," mocking the story and its creator. Caliborn's aim is to undermine and ridicule everything, from the protagonist's struggles to the art itself. Despite his effort to hijack the narrative, Caliborn unwittingly contributes to the story's continuation, as items left by The Author lead him to unlock new content, extending Homestuck even as he tries to derail it.

In the main narrative, the players have just arrived to the new session. The scattered kids face challenges under the Condesce's plan to take over their session and its nascent Universe, with Jade and Jane falling under her control, and Jake, Roxy and Dirk imprisoned or otherwise AWOL. 

Aranea uses the Ring of Life juju to come back to life. After a massive battle, where most characters die due to the Condesce and Aranea's meddling, John meets his Denizen in desperation, and gains a power to jump across the canon of the narrative itself. Directed by a dying Terezi's instructions, John "retcons" events so that Vriska survives instead of dying. Vriska takes an active role in helping the group strategize for the upcoming showdown.

In the final ACT ACT ACT, Caliborn shows a "spoiler to the ending of things" in Claymation form. In his tale, eight kids show up to defeat him, and four of them (which seem to be some version of John, Rose, Dave and Jade) are trapped inside a house-shaped juju. The four remaining combatants, implied to be the Alpha Kids, cause the ultimate defeat of Caliborn, but also the creation of Lord English. The ultimate villain is revealed to be a product of the combined essences of Caliborn, Arquiusprite, and Gamzee, created as an unwitting side effect of one Dirk's attack. A combination of their souls is trapped inside the Lil Cal juju, which is (only temporally, as we've previously seen) sent into the Void by one Roxy.

In the present time and session, and with proper planning, an even more intense battle ensues. It involves all of the main players, Spades Slick, Lord Jack, and The Condesce. Roxy manages to land a killing blow on The Condesce, and Dave and Dirk defeat the Jacks. Avoiding all casualties, the players reunite at the victory lilypad, where Calliope is resurrected, finally able to take her place in the waking world.

After a long sequence of post-battle conversations, the players use the power of the Forge and the Genesis Frog create a new universe, Universe C. In the Furthest Ring, the alternate Calliope, the Muse of Space, helps collapse the Green Sun into a Black Hole, where Vriska deploys the secret weapon against Lord English, a house-shaped juju, but his defeat is not shown.

The surviving players and their allies pass through the door to claim the Ultimate Reward: a new world for themselves, where the previous Earth B (now C) is placed and prepared to support new life. The players are crowned as the creators of this new universe, and they begin to rebuild civilization. The story concludes with a hint that John still needs to return to finish the job and defeat Caliborn, a closed loop that will end with Lord English's creation."""

# Taken from page descriptions, better than nothing
A6A6A6_SUMMARY = """Jane and the Nannas are ready to heal. Jasprose’s windows will move the healers between the fights in the session. Jane can resurrect each fighter once.
Davepeta, Aradia, and Sollux find the ghost army. Karkat moves first, charging fearlessly into Lord English’s double death beam. Karkat wakes up on Lofaf.
Strife begins on Derse. Rose, John, and Roxy play fraymotifs to clobber Condy with a Zazzerpan statue, whip her with wind, and pummel her with perfectly generic objects. Condy fights back with all the psionic powers that will avail her.
Strife begins on Lomax. Itchy and Stitch get the first hits in. Jake shoots Sawbuck and time-travels to several other moments in the fight. Arquiusprite joins, taunting Cans with quadrupedal automata and the fridge in which Gamzee is still locked. Cans clocks Jake, Arquius, and several Felt to Lofaf, where Clover humiliates Karkat, and again back to Lomax. Crowbar shoots and kills Jake.
The healing squad goes first to Derse, where Rose and Kanaya have died. Jane resurrects them while Jasprose distracts Condy. The fight on Derse reaches its most chaotic and madcap when Cans clocks Arquius and the fridge into it. Next, the healers go to Lomax and resurrect Jake. Everyone loves Nanna’s cookies. Jane stays to fight aside Jake. Casey von Salamancer’s skeleton army overruns the combatants.
Strife begins on Lotak, first between the Jacks. The three sides fight indiscriminately. Dave and Terezi play a fraymotif to anticipate and avert bad outcomes. Dirk tries to rip Lord Jack’s soul, but the Archagent is already too ripped with clockwork majyyks. Cans clocks Arquius and the fridge into their fight, then clocks Arquius back to Lofaf, leaving the fridge on Lotak.
Arquius and Cans end up on Derse in time to see Dad Crocker break out of jail. Dad pies Cans.
Strife begins in the ruin of Prospit between the chess dogs and in the bubble between Vriska and the Author.
Caliborn defeats Yaldabaoth and finds his reward: a judgment clock and a chest containing the weapon he will use in his masterpiece.
The Muse of Space reaches the Green Sun.
Serenity finds John on Derse, Jade on Prospit, Dave on Lotak, and Karkat on Lofaf, filling them with resolve. Finally, Serenity is reunited with the Vagabond.
Strife begins in the bubble between the ghost army and the constant conqueror. Lord English annihilates many ghosts. Meenah, Tavros, Aradia, and Davepeta put up a fight.
John, Roxy, Rose, and Kanaya regroup against Condy, who has choked Jane. Dave, Dirk, and Terezi renew the fight against the Jacks. They all take fearsome beatings, but they have come so far that they will not give up now.
Gamzee has been rattled around in the Crocker family fridge to the point of tears.
Jake, Karkat, and Dad Crocker vanquish The Felt.
To stop the chess dogs fighting, Jade teleports them from place to place and puts herself between them. They still have Bec’s loyalty and love of her. This may keep them from stabbing her, but not from using non-lethal methods. The Monarch punches Jade out and resumes her quest to neutralize the Slayer.
The Archagent gets Dirk in a hold with his crowbar. Spades Slick gets the Archagent in a hold with his cuestaff. Their heads are all lined up, so Dave takes the shot. He decapitates them and breaks the unbreakable katana in one blow. Then, for the first time since his old session, Dave uses time travel. He recovers Dirk’s head and body and evacuates Terezi from Lotak. Their enemies may be dead, but the Archagent’s cursed clockwork head is a bomb. It implodes into a black hole, taking the fridge and most of Lotak with it.
The Monarch cuts off the Slayer’s arm. He loses his ring and is Bec Noir no more. He gets the non-lethal methods too. He lands unconscious next to Jade.
The Batterwitch makes the fatal mistake of turning her back on Roxy, who impales her on Bro’s sword. Ding dong.
Nanna heals Jane and meets Dad Crocker. The Derse and Lotak teams return to the lilypad. Jane resurrects Dirk.
Jade wakes. Callie has made a new friend, and Jack Noir perhaps a new sweetheart, in Ms. Paint.
The Author, done hassling Vriska, gives Lord English the big thumbs down.
The Monarch finds the Vagabond and takes off her Ring of Orbs Fourfold. She wants both rings to be unmade in the Forge.
Jake, Karkat, Jade, and Callie come to the lilypad. They have joyful reunions all round.
Dad Crocker meets John, whose post-scratch self was Dad’s father.
The Monarch and Vagabond climb to the lip of the Forge."""

# Taken from page descriptions, better than nothing
ACT_7_SUMMARY = """The Rings of Orbs Fourfold light the Forge, propelling the genesis tadpole into Skaia. The tadpole begins to metamorphose and dives into the hole in the Battlefield by which John excavated The Tumor. There, the tadpole completes its metamorphosis. It eats the Battlefield and becomes the splendid Genesis Frog, embodiment of a new universe.
The Muse of Space makes a black hole next to the Green Sun. It eats the Sun and the surrounding space of the Furthest Ring. Aradia gets her wish to see this whole place break apart.
Vriska is ready to use her weapon. She opens the chest and produces an image of a house which contains the aspects of Breath, Light, Time, and Space. This artefact does something ambiguous to Lord English.
In Yaldabaoth’s lair, Caliborn smashes the judgment clock and receives the clockwork majyyks.
A door to the ultimate reward appears before those assembled on the lilypad. They will take Earth through to their new universe and terraform it. The ocean which covered Earth will recede and terrestrial life will return. Carapacians will live on Earth with the victors, fulfilling the promise of Can Town.
John reaches for the doorknob.
In snapshots from Earth C, the victors return to the lab meteor which crashed into Earth. They go to the ectobiology lab and make human, troll, and carapacian babies. They leave these babies in the care of mature carapacians, then time travel forward to live in a later stage of the resulting civilization.
They are hailed as the “creators” and proclaimed kings and queens:
John and Jane are crowned in the Human Kingdom, accompanied by Dad and Nannasprite.
Karkat, Terezi, Dave, and Jade are crowned in the Troll Kingdom.
Rose, Roxy, Kanaya, and Callie are crowned in the Carapacian Kingdom.
Jake and Dirk are crowned in the Consort Kingdom, accompanied by GCatavrosprite.
John celebrates his seventeenth birthday. Rose and Kanaya marry. Dave, Karkat, and Jade excavate shitty Liberties. Dirk makes a skull-monster–shaped sparring partner.
By the time John turns eighteen, Terezi has left Universe C to look for Vriska in the disintegrating Furthest Ring. Her search seems fruitless.
Jane remakes CrockerCorp. Jake remakes SkaiaNet, though its only seeming function is to get his ass on television. Kanaya oversees the Mother Grub and her wiggler broods.
On his nineteenth and twentieth birthdays, John is increasingly solitary. Finally Caliborn sends him a snap spoiling for a fight. This may be what provokes the battle depicted in Caliborn’s masterpiece."""

part_urls = ["part-1", "part-2", "part-3", "part-4"]

act_urls_per_part = {
    "part-1": ['act-1', 'act-2', 'act-3', 'intermission', 'act-4'],
    "part-2": ['a5a1', 'a5a2', 'eoa5'],
    "part-3": ['a6-1', 'a6-2', 'a6-3', 'a6-4', 'a6-5'],
    "part-4": ['a6a6-1', 'a6a6-2', 'a6a6-3', 'a6a6-4', 'a6a6-5', 'a6a6a6', 'act-7']
}


def _clean_cell_html(cell: str) -> str:
    # Acts and pages has the ugly habit of prefixing and affixing each cell with a bunch of \n linebreaks
    if not cell:
        return cell
    # Trim multiple \ns from the start
    left_strip = cell.lstrip('\n')
    # Trim multiple \ns from the end
    right_strip = left_strip.rstrip('\n')
    return right_strip


part_summaries = {}
act_summaries = {}


def crawl_acts_and_pages(url: str) -> tuple[dict, dict, list]:
    acts_and_pages = []
    part_summaries = {}
    act_summaries = {}

    for part_url in part_urls:
        # Crawl part description, located in <table class="part">, first row, first column
        part_link = f"{url}{part_url}"
        part_response = requests.get(part_link)
        part_soup = BeautifulSoup(part_response.content, 'html.parser')

        part_table = part_soup.find('table', class_='part')
        if part_table is None:
            continue
        part_td_elements = part_table.find_all('td')

        if len(part_td_elements) < 1:
            continue

        part_summary = part_td_elements[0].get_text().strip() if part_url != 'part-4' else PART_FOUR_SUMMARY
        part_summaries[part_url] = part_summary

        for act in act_urls_per_part[part_url]:
            # Crawl act synopses, there are multiple synopses per act, organized per viewpoint or theme
            # Each is stored in one <tr class="synopsis">, with the first <th> being the viewpoint/theme,
            # we will make it allcaps before storing it in an array in the right key of the act_summaries dict
            act_link = f"{url}{act}"
            act_response = requests.get(act_link)
            act_soup = BeautifulSoup(act_response.content, 'html.parser')
            act_synopses = act_soup.find_all('tr', class_='synopsis')
            if act == 'eoa5':
                # rafe erroneously called eoa5/intermission 2 a different act, it should just be a5a2
                # Grab it from a5a2, which should be processed right before this
                act_summary = act_summaries['a5a2']
            elif act == 'a6a6a6':
                act_summary = [["GENERAL", A6A6A6_SUMMARY]]
            elif act == 'act-7':
                act_summary = [["GENERAL", ACT_7_SUMMARY]]
            else:
                act_summary = []
                for synopsis in act_synopses:
                    viewpoint = synopsis.find('th').get_text().upper()
                    if not viewpoint:
                        viewpoint = "GENERAL"
                    summary = synopsis.find('td').get_text().strip()
                    act_summary.append([viewpoint, summary])
            act_summaries[act] = act_summary

            # And now we crawl the page data in the act pages
            # Iterate over all tables that are not the sidebar or header navigation
            page_data_tables = [
                table for table in act_soup.find_all('table')
                if table.get('id') not in ['sidebar-nav', 'header-nav']
            ]

            for page_data_table in page_data_tables:
                page_data_rows = page_data_table.find_all('tr')
                for row_index, row in enumerate(page_data_rows):
                    # get rows that have at least four <td> elements or contain a table with class x2
                    cells = row.find_all('td')
                    if len(cells) < 4 and not row.find('table', class_='x2'):
                        continue
                    page_links = cells[-1].find_all('a')
                    # extract the PAGE_NUMBER from the a hrefs
                    page_numbers = [link['href'].split('/')[-1] for link in page_links]
                    # extract the HTML summary from the third column
                    page_summary = cells[2].get_text().strip()
                    # Handle x2 table data if present
                    x2_table = cells[2].find('table', class_='x2')
                    if x2_table:
                        # Remove the x2 table from the original summary
                        x2_table.extract()
                        page_summary = cells[2].get_text().strip()
                        x2_rows = x2_table.find_all('tr')
                        for x2_row in x2_rows:
                            x2_cells = x2_row.find_all('td')
                            if len(x2_cells) == 2:
                                x2_content = f" {x2_cells[0].get_text().strip()} | {x2_cells[1].get_text().strip()}"
                                page_summary += x2_content
                    # save everything per page
                    if act == 'eoa5':
                        act = 'a5a2'
                    for page_number in page_numbers:
                        acts_and_pages.append({
                            'page_id': page_number,
                            'page_group_summary': page_summary,
                            'act': act,
                            'part': part_url
                        })

    return part_summaries, act_summaries, acts_and_pages


# Let's check if we already have the files for parts, acts, and pages
if os.path.exists(ACTS_AND_PAGES_PART_SUMMARIES) and os.path.exists(ACTS_AND_PAGES_ACT_SUMMARIES) and os.path.exists(
        ACTS_AND_PAGES_PAGE_SUMMARIES) and not FORCE_ACTS_CRAWL:
    with open(ACTS_AND_PAGES_PART_SUMMARIES, 'r', encoding='utf-8') as f:
        part_summaries = json.load(f)
    with open(ACTS_AND_PAGES_ACT_SUMMARIES, 'r', encoding='utf-8') as f:
        act_summaries = json.load(f)
    with open(ACTS_AND_PAGES_PAGE_SUMMARIES, 'r', encoding='utf-8') as f:
        page_summaries = json.load(f)

    print("Acts and Pages data loaded from files.")

else:
    print("Acts and Pages data not found. Crawling website...")

    # Crawl the website to extract the Acts and Pages data
    part_summaries, act_summaries, page_summaries = crawl_acts_and_pages(ACTS_AND_PAGES_URL)

    # Save the extracted data to JSON files
    with open(ACTS_AND_PAGES_PART_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(part_summaries, f)
    with open(ACTS_AND_PAGES_ACT_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(act_summaries, f)
    with open(ACTS_AND_PAGES_PAGE_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(page_summaries, f)

    print("Acts and Pages data saved to files.")

Acts and Pages data loaded from files.


In [22]:
part_summaries_df = pd.DataFrame(part_summaries.items(), columns=['part', 'part_summary'])
part_summaries_df

,part,part_summary
0,part-1,"John, Rose, Dave, and Jade are online friends...."
1,part-2,"Aradia, Tavros, Sollux, Karkat, Nepeta, Kanaya..."
2,part-3,"In the first session, Skaia sent the cloned ba..."
3,part-4,Caliborn seizes control of the narrative. He r...


In [23]:
act_summaries_df = pd.DataFrame(act_summaries.items(), columns=['act', 'act_summary'])
act_summaries_df

,act,act_summary
0,act-1,"[[JOHN, Homestuck begins as the story of four ..."
1,act-2,"[[THE MEDIUM, John narrowly escapes fiery deat..."
2,act-3,"[[JADE, Our fourth kid is the sleepy Jade Harl..."
3,intermission,"[[THE MANSION, Spades Slick, Diamonds Droog, C..."
4,act-4,"[[KIDS, Previously: John reached the First Gat..."
5,a5a1,"[[SGRUB, Previously, the aliens that were trol..."
6,a5a2,"[[GENERAL, This synopsis covers Act 5 Act 2, C..."
7,eoa5,"[[GENERAL, This synopsis covers Act 5 Act 2, C..."
8,a6-1,"[[JANE, Act 6 begins on Earth on the flip side..."
9,a6-2,"[[TWO DOWN, After Jane gets knocked out at the..."


In [24]:
page_summaries_df = pd.DataFrame(page_summaries)
page_summaries_df.sample(5)

,page_id,page_group_summary,act,part
4190,4193,Auto Responder pesters Jake about Brobot.,a6-1,part-3
4128,4131,Jane waits for the mail. Jake pesters her abou...,a6-1,part-3
1722,1722,Dream Rose tosses Lil Cal out of Dave’s tower....,act-4,part-1
3913,3916,Scratch repairs the photo album.\n\nIn any ord...,a5a2,part-2
6932,6932,Act 6 Act 6 Act 4.\n \nCaliborn suffers a cr...,a6a6-4,part-4


In [25]:
# Let's check every page number and see if we have a row for it, print the missing page numbers
# First page is "1" and last page is "8130"
# (this ignores secret pages and only addresses digit strings, that's okay)
for i in range(1, 8131):
    if str(i) not in page_summaries_df['page_id'].values:
        print(f"Missing page: {i}")

# If everything works fine we should only be missing [2399, 3038, 3088, 7902, 7903, 7904], the Six Official Missing Pages

Missing page: 2399
Missing page: 3038
Missing page: 3088
Missing page: 7902
Missing page: 7903
Missing page: 7904


In [26]:
# We can merge the page summaries alone into mspa_df_with_pov, since putting the act/part transcripts on every single row would be a bit much
# We'll merge on vizPageNumber and page_id
# We need to convert mspa_df_with_pov['vizPageNumber'] to a string to merge
mspa_df_with_pov['vizPageNumber'] = mspa_df_with_pov['vizPageNumber'].astype(str)

# Rename the page_id column to vizPageNumber
page_summaries_df.rename(columns={'page_id': 'vizPageNumber'}, inplace=True)

# Merge the data into mspa_df_with_pov
mspa_df_with_pov_and_pages = mspa_df_with_pov.merge(page_summaries_df, on='vizPageNumber', how='left')

# Show some data with page summaries
mspa_df_with_pov_and_pages[mspa_df_with_pov_and_pages['page_group_summary'].notnull()].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part
1752,story,001478,Next.,,1229226888,/advimgs/ps/ps1265.gif,[001479],001477,retro,[],...,NaN,NaN,NaN,STRIPPER 1203,[],[],[],Droog gets the drop on Stitch.,intermission,part-1
8184,story,006901,==>,"|DIALOGLOG|<br /><span style=""color: #005682"">...",1338513604,/storyfiles/hs2/04998.gif,[006902],006900,,[],...,NaN,NaN,NaN,NaN,[],"[Aranea, Dave, Dirk, Jake, Kanaya, Karkat, Mee...","[[Aranea: Visit Jake, Aranea: Notice Jake's fl...","Her story angers Aranea, who tells their post-...",a6-3,part-3
5049,story,004074,======>,"|PESTERLOG|<br /><span style=""color: #005682"">...",1278458824,/storyfiles/hs2/02174.gif,[004075],004073,,[],...,NaN,NaN,NaN,NaN,[],"[Aradia, Tavros, Vriska]","[[Aradia: FLARP], [Tavros: Go Flarping], [Vris...",Perigees prior: Tavros plays Flarp. Vriska fuc...,a5a1,part-2
10279,story,008833,AND I WEEP THE TEARS.,,1414892592,/storyfiles/hs2/06930.gif,[008834],008832,A6A6,[],...,NaN,NaN,NaN,NaN,[],[Caliborn],[[Caliborn: Return to basics]],Act 6 Act 6 Act 4.\n \nCaliborn suffers a cr...,a6a6-4,part-4
10825,story,009368,[A6A6I5] ====>,"|DIALOGLOG|<br /><span style=""color: #ff6ff2"">...",1430257434,/storyfiles/hs2/07464.gif,[009369],009367,,[],...,NaN,NaN,NaN,NaN,[],"[Equiusprite, Dave, Jake, John, Kanaya, Karkat...",[[Equiusprite: Attempt to remove tiaratop from...,John and Roxy arrive.,a6a6-5,part-4
3743,story,003057,[I] SS: Captchalogue carriage clock.,You obviously have no idea what that means.<br...,1263629544,/storyfiles/hs2/01157.gif,[003058],003056,,[],...,NaN,NaN,NaN,\nWEAPONS\n\nITEM\n,"[Part 1, Intermission, Felt Manor, Stairs, Mid...",[Spades Slick],"[[Spades Slick: Enter mansion, Spades Slick: C...",Spades Slick starts wrecking clocks.,intermission,part-1
2915,story,002388,John: Bring 2 cans of shaving cream.,You captchalogue two cans of SHAVING CREAM jus...,1250457790,/storyfiles/hs2/00488_2.gif,[002389],002387,,[],...,NaN,NaN,NaN,NaN,"[Act 2, Part 1, John's House, Piano, Cake, Fak...",[John],"[[John: Put bunny in strife specibus, John: St...",Rose builds John’s house upwards.,act-2,part-1
3197,story,002621,[S] WV: Lead your men to victory!,You waste more than four hours on this tomfool...,1254443810,/storyfiles/hs2/00721/00721.swf,[002622],002620,,[F],...,NaN,NaN,NaN,NaN,"[Part 1, Act 2]",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]",Cinematic 15: To Victory!,act-2,part-1
6409,story,005318,==>,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",1296610898,/storyfiles/hs2/03415.gif,[005319],005317,,[],...,NaN,NaN,NaN,BEEEEEEEEEE-,[],"[Jade, John]","[[Jade: Talk with John, Jade: Ask John how he ...",John lets the Vassal keep the ring and catches...,a5a2,part-2
8519,story,007218,[A6I3] ==>,,1348706732,/storyfiles/hs2/05315.gif,[007219],007217,,[],...,NaN,NaN,NaN,NaN,[],"[Jack Noir, Parcel Mistress]","[[Jack Noir: Be chased by PM], [Parcel Mistres...",Noir enters a bubble of his rampage on the Bat...,a6-3,part-3


# Adding MSPA commentary data

From the books, we have commentary for groups of pages in Problem Sleuth and Homestuck that might be a bit pointless for prediction models, but is still useful text data.

In [27]:
"""
MSPA_COMMENTARY is a json export of an SQL table that contains the per-Viz-page commentary (by the creator Andrew Hussie) and notes (by me and other volunteers, like fanmade commentary).
There are two tables, one for Homestuck ("commentary") and another for Problem Sleuth ("problem-sleuth") with this format:
```json
[
{
    "type": "table",
    "name": "story",
    "database": "commentary",
    "data": 
[
    {
        "id":"1",
        "title":"Homestuck",
        "page":"7",
        "commentary":"Homestuck (the stable release) began on April 13th (i.e. 4\/13), 2009. My intent was always to make the start date a very significant number in the story, recurring frequently. Consequently, I decided to make him thirteen years old, thus making the story about four thirteen-year-old kids. There are more references to this number than can be mentioned casually, some of which are serendipitous. Playing cards, which have a good deal of relevance later, are comprised of four suits of thirteen cards each, for instance.",
        "notes":null
     },
...
{
    "type": "table",
    "name": "problem-sleuth",
    "database": "commentary",
    "data": 
[
{
    "id": "1",
    "title": "Problem Sleuth",
    "page": "15",
    "commentary": "The title \"Problem Sleuth\" resulted from a quest to determine the most awkward and peculiar term for a detective imaginable. I ran a bunch of other possibilities by my friend, and she instantly green-lighted PS when I said it. Little did I know I would find a way to top that phrase when introducing the tall skinny one a few hundred pages later. Though his name probably wouldn't have made as good a title, admittedly.",
    "notes": null
},
...
```
It should be fairly trivial to turn this into a dataset that we can merge based on the adventure title ("vizStory") and page number ("id"->"vizPageNumber", the "page" is the physical book page). Then, we can keep the page_commentary and page_notes columns.
"""

# Load the data
with open(MSPA_COMMENTARY, 'r', encoding="utf-8") as f:
    commentary_data = json.load(f)

hs_data = commentary_data[0]['data']
ps_data = commentary_data[1]['data']

# Create DataFrames
hs_df = pd.DataFrame(hs_data)
ps_df = pd.DataFrame(ps_data)

# Fill the vizStory column with each title on each dataframe
hs_df['vizStory'] = 'Homestuck'
ps_df['vizStory'] = 'Problem Sleuth'

# Concatenate the DataFrames
commentary_df = pd.concat([hs_df, ps_df], ignore_index=True)

# We don't care about the title or the page (book page), we care about id (renamed to vizPageNumber), the commentary and notes
commentary_df = commentary_df[['vizStory', 'id', 'commentary', 'notes']]
commentary_df.rename(columns={'id': 'vizPageNumber', 'commentary': 'pageCommentary', 'notes': 'pageNotes'},
                     inplace=True)

# Show some data
commentary_df.sample(10)

,vizStory,vizPageNumber,pageCommentary,pageNotes
3181,Homestuck,3183,None,None
706,Homestuck,707,None,None
3641,Homestuck,3643,None,None
4792,Problem Sleuth,1080,None,None
4903,Problem Sleuth,1191,None,None
1495,Homestuck,1497,None,None
1359,Homestuck,1361,None,None
2902,Homestuck,2904,Rose is tucked away in some completely unknown...,None
126,Homestuck,127,None,None
597,Homestuck,598,None,None


In [28]:
# Merge the data into mspa_df_with_pov
mspa_df_with_pov_and_commentary = mspa_df_with_pov_and_pages.merge(commentary_df, on=['vizStory', 'vizPageNumber'],
                                                                   how='left')

# Show some data with notes
mspa_df_with_pov_and_commentary[mspa_df_with_pov_and_commentary['pageNotes'].notnull()].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,pageCommentary,pageNotes
3111,story,002551,[S] John: Wake up.,,1253223641,/storyfiles/hs2/00651/00651.swf,[002552],002550,,"[F, HQ, BOLIN, BOLINHQ]",...,NaN,NaN,"[Part 1, Act 2]",[John],"[[John: Wake up, John: Prepare for boss battle]]",Cinematic 13: John Wakes.\nJade pesters John a...,act-2,part-1,John was probably way more excited by the fluf...,Makin: This flash originally used the last par...
6748,story,005614,Exit.,,1302748513,/storyfiles/hs2/03711/03711.swf,[005615],005613,,[F],...,NaN,NaN,[],[Terezi],"[[Terezi: Find Nepeta's body, Terezi: Eject di...",Terezi dwells on the necessity of taking Vrisk...,a5a2,part-2,That's it for exactly two years of Homestuck. ...,Makin: That's it for the commentary mod of the...
3314,story,002722,[S] Jade: Play a hauntingly relaxing bassline.,,1256635621,/storyfiles/hs2/00822/00822.swf,[002723],002721,,"[F, BOLIN]",...,NaN,NaN,"[Part 1, Act 3]",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",Cinematic 18: Hauntingly Relaxing Bassline.,act-3,part-1,Here we confirm that Jade lives on that island...,"Makin: This flash originally used the song ""Gu..."
3325,story,002730,[S] Jade: Open FreshJamz!,You open the FRESHJAMZ MEDIA PLAYER and add Da...,1256977886,/storyfiles/hs2/00830/00830.swf,[002731],002729,,"[F, BOLIN]",...,NaN,NaN,"[Part 1, Act 3]",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",Pseudogame 9: FreshJamz.,act-3,part-1,"With the (playable!) FRESHJAMZ playlist, we ar...","Makin: This flash originally used the songs ""R..."
2369,story,001924,John: Examine incoming message.,You pull up to your COMPUTER. This is where yo...,1239779693,/storyfiles/hs2/00024_2.gif,[001925],001923,,[],...,NaN,\n[SYSTEM]\n\n[TYPHEUS]\n\n[PESTERCHUM]\n\n[pf...,"[Part 1, Act 1, John's House, John's Room, Com...",[John],"[[John: Be introduced, John: Mess around]]",Dave pesters John about loot and apple juice.,act-1,part-1,None,Makin: The ~ATH file on John's desktop actuall...
3339,story,002743,[S] Jade: Descend.,"Try as you might, you can't stop your mind fro...",1257722088,/storyfiles/hs2/00843/00843.swf,[002744],002742,,"[F, HQ]",...,NaN,NaN,"[Part 1, Act 3]","[Jade, John]","[[Jade: <==, Jade: Talk with Rose, Jade: Infor...","Cinematic 21: Strife with the Crude Ogres, Adv...",act-3,part-1,The particular configuration of reminders she'...,Makin: The tire swing attack Hussie mentioned ...
3499,story,002880,[S] Jade: Retrieve package.,,1260259150,/storyfiles/hs2/00980/00980.swf,[002881],002879,,"[F, HQ, BOLIN, BOLINHQ]",...,NaN,NaN,"[Part 1, Act 3]","[Becquerel, Jade]","[[Becquerel: Play with Jade, Becquerel: Tuck J...",Pseudogame 12: Strife with Bec.,act-3,part-1,"0:08 - Jade: Seriously, just walk over there a...","Makin: This flash originally used the song ""Mu..."
3551,story,002922,Rose: Build as much as you can as fast as you ...,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",1261008164,/storyfiles/hs2/01022.gif,[002923],002921,,[],...,NaN,NaN,"[Part 1, Act 3, John Egbert, John's House, Alc...","[John, Rose]","[[John: Start building up], [Rose: Build up, R...",Rose pesters John about brain problems.,act-3,part-1,"In the grand scheme of things, I wonder which ...","Drew: Unfortunately, I think we have the answe..."
4604,story,003743,Dave: Combine fetus in a jar and self portrait...,That would apparently make DAVE'S BRAIN IN A J...,1273812843,/storyfiles/hs2/01843_1.gif,[003744],003742,,[],...,NaN,"\nDAVE'S BRAIN IN A JAR\n? 1,000,000,000\n","[Part 1, Act 4, Dave's Brain in a jar, Holopad...",[Dave],[[Dave: Wake up and jam.]],"Dave copies and reads Rose’s notebooks, then m...",act-4,part-1,I don't remember if we ever found out what typ...,Drew: After all these years the mystery has be...
1438,story,001214,HD: Turn crank on scale valve.,The flow amplifies.,1224718031,/advimgs/ps/ps1001.gif,[001215],001213,retro,[],...,NaN,NaN,[],[],[],

# Cleaning up

We plan on converting this to CSV and potentially JSONL, so right now have a lot of object columns that should be strings, a bunch of empty strings and NaNs that should be pd.NAs. 

In [29]:
# mspa_df_with_pov currently has these columns representing MSPA data:
# section (str, per page): the section of the MSPA website the page is in, always a string like "story"
# pageId (str, per page): the page number, usually a six digit padded string like "001901", sometimes a regular string like "pony"
# title (str, per page): the title of the page, usually the command given in the previous page, always a string like "John: Enter name."
# content (str, per page): the content of the page, usually the page minus the panels (includes dialog and HTML)
# timestamp (timestamp, per page): the UNIX timestamp of the page, always a number like 1580000000
# src (str, per panel): the partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like "/storyfiles/hs2/001901.gif"
# next (list of strings, per page): the pageId of the next pages, usually a list of one string like ["001902"]
# previous (string, per page): the pageId of the previous page, usually a string like "001900"
# theme (str, per page): the theme of the page, like "A6A6", always a string, usually empty
# flag (list of strings, per page): a list of flags for Homestuck Collection presentation, some flags like ["R1"] show that the page is part of a "retcon" where Hussie went back and edited panels, others like ["FIREFLY"] show the page has special javascript that shows fireflies flying around the page, a page usually has no flag
# vizStory (str, per page): essentially a readable version of the section, always a string like "Homestuck"
# vizPageNumber (str, per page): pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ, always a non-padded string since string pages like "pony" still exist
# vizPageId (str, per page): we should remove this column, being a combination of section number and viz page that was only used in one dataset we wanted to merge with
# vizLink (str, per page): the link to the page in the VIZ website, always a full URI like "https://homestuck.com/story/5320"
# pageNumber (str, per page): like vizPageId but for MSPA format, also should be removed
# transcript class (str, per panel): the class of the transcript, like "dialogue" or "narration", always a string, usually a NaN which should be replaced with pd.NA
# transcript style (str, per panel): the style of the transcript, in CSS, like "font-weight: bold", always a string, usually a NaN which should be replaced with pd.NA
# transcript tag (str, per panel): the tag of the transcript, like "p" or "div", always a string, usually a NaN which should be replaced with pd.NA
# transcript (str, per panel): the transcript of the panel, the text that appears in the panel, always a string, usually a NaN which should be replaced with pd.NA
# tags (list of strings, per page): the tags of the page, like ["John Egbert", "Rose Lalonde", "Land of Wind and Shade"], always a list of strings, usually an empty list
# pov_characters (list of strings, per page): the characters that appear in the page according to the POV cam, always a list of strings, usually an empty list
# pov_commands (list of strings, per page): the commands that the characters have in the page according to the POV cam, always a list of lists of strings (per character), usually an empty list
# pageCommentary (string, per page): the commentary of the page, usually by Andrew Hussie, always a string, usually a NaN or None which should be replaced with pd.NA
# pageNotes (string, per page): the notes of the page, usually by volunteers, always a string, usually a NaN or None which should be replaced with pd.NA
# page_group_summary (string, per group of pages): the summary of an arbitrary but small group of pages, or a scene, as decided and described by Rafe from rafe.name, always a string, should be replaced with pd.NA if empty
# act (str, per page): the act the page is in, always a string like "act-1"
# part (str, per page): the part the page is in, always a string like "part-1"

# Let's start by removing the columns we don't need
mspa_df_cleaned = mspa_df_with_pov_and_commentary.drop(columns=['vizPageId', 'pageNumber'])

# Replace NaN with pd.NA in the transcript columns
na_to_pdna_columns = ['transcript_class', 'transcript_style', 'transcript_tag', 'transcript', 'pageCommentary',
                      'pageNotes', 'page_group_summary', 'act', 'part']
mspa_df_cleaned[na_to_pdna_columns] = mspa_df_cleaned[na_to_pdna_columns].where(
    mspa_df_cleaned[na_to_pdna_columns].notnull(), pd.NA)

# Replace empty strings with pd.NA in all columns
mspa_df_cleaned = mspa_df_cleaned.replace('', pd.NA)

# Show some cleaned data
mspa_df_cleaned.sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,pageCommentary,pageNotes
9237,story,007832,[A6I5] ==>,"|DIALOGLOG|<br /><span style=""color: #626262"">...",1361088765,/storyfiles/hs2/05929.gif,[007833],007831,<NA>,[],...,<NA>,<NA>,[],"[Dave, Karkat]","[[Dave: Prepare for arrival in new session], [...",Karkat trolls Dave about their approach.,a6-5,part-3,<NA>,<NA>
4194,story,003415,==>,You then pay the steep fee of 1000 BUILD GRIST...,1267400126,/storyfiles/hs2/01515.gif,[003416],003414,<NA>,[],...,<NA>,⬡ 1798/2000 → 798,"[Side 1, Part 1, Act 4, Cruxite Dowel, Alchemi...","[Dave, Jade]","[[Dave: EGG!], [Jade: EGG!]]",Jade adds a jumper block extension to Dave’s a...,act-4,part-1,<NA>,<NA>
8365,story,007065,==>,<NA>,1340699641,/storyfiles/hs2/05162.gif,[007066],007064,<NA>,[],...,<NA>,<NA>,[],[],[],Caliborn jeers Calliope about the clusterfuck:...,a6-3,part-3,<NA>,<NA>
11178,story,009716,[A6A6I5] ====>,"|DIALOGLOG|<br /><span style=""color: #f141ef"">...",1436139970,/storyfiles/hs2/07812.gif,[009717],009715,<NA>,[],...,<NA>,<NA>,[],"[Jake, Jasprosesprite^2, Tavrosprite]","[[Jake: <====, Jake: Nickname Tavvy, Jake: Be ...",Jasprosesprite² brings Nepeta’s head to Lomax ...,a6a6-5,part-4,<NA>,<NA>
11052,story,009596,[A6A6I5] ====>,<NA>,1435011254,/storyfiles/hs2/07692/07692.swf,[009597],009595,<NA>,[F],...,div,<strike>CHOOSE YOUR CHARACTERS!</strike>,[],"[Jane, John, Rose, Terezi]","[[Jane: Character select], [John: Character se...",Rose worries about being OG. John starts to no...,a6a6-5,part-4,<NA>,<NA>
4836,story,003907,Karkat: Check out magazine.,It's the latest issue of GAME GRUB.<br /><br /...,1276553324,/storyfiles/hs2/02007.gif,[003908],003906,<NA>,[],...,<NA>,\n}}BURG\nEMAG{{\n\n}}SKAEL\nEVISULCXE{{\n,[],[Karkat],"[[Karkat: Be introduced, Karkat: Talk to Gamzee]]",Karkat takes in the view outside. Gamzee troll...,a5a1,part-2,<NA>,<NA>
6690,story,005564,==>,<NA>,1300504917,/storyfiles/hs2/03661.gif,[005565],005563,<NA>,[],...,<NA>,10:25:00 → :24:59 → :58 → :57 → :56 → :55 → :5...,[],[John],"[[John: Locate Tumor, John: Meet CD, John: Reu...","In the core of the Battlefield, John finds a m...",a5a2,part-2,"We see that the Tumor has a huge, magic 8-ball...",<NA>
79,story,000083,Check that door to see if it is locked.,Your curiousity about the door gets the better...,1180937455,/advimgs/jb/lv2_option2/mspaintadventure05b.gif,[000084],000082,retro,[],...,<NA>,TUMBLE,[],[],[],Cinematic 2: Homestuck.\nThe doll loses an arm.,act-1,part-1,<NA>,<NA>
2451,story,001991,John: Retrieve the package and flee to your room!,You cannot ABSCOND! This pesky GUARDIAN is blo...,1240978551,/storyfiles/hs2/00091.gif,[001992],001990,<NA>,[],...,<NA>,> ABSCOND,"[Part 1, Act 1, John's House, Kitchen, John Eg...","[Dad, John]","[[Dad: Finish cake, Dad: Strife!, Dad: Gain pr...",Dad lavishes John with baked goods.,act-1,part-1,I think we are all at ease with the implicatio...,<NA>
7263,story,006039,Jane: Look around.,Yep. This one changed too. Crockercorp is noth...,1321253547,/storyfiles/hs2/04136_1.gif,[006040],006038,<NA>,[],...,<NA>,<NA>,[],[Jane],[[Jane: <==]],Crockercorp adopts a more imperious iconograph...,a6-1,part-3,<NA>,<NA>


In [30]:
# Fix the inconsistent casing (some columns are snake_case, others camelCase)
# Annoyingly, there doesn't seem to be a standard for this for ML datasets, so we'll go with Python's snake_case
# This means altering pageId, vizStory, vizPageNumber, vizLink, pageCommentary, pageNotes

mspa_df_cleaned.rename(columns={
    'pageId': 'page_id',
    'vizStory': 'viz_story',
    'vizPageNumber': 'viz_page_number',
    'vizLink': 'viz_link',
    'pageCommentary': 'page_commentary',
    'pageNotes': 'page_notes'
}, inplace=True)

mspa_df_cleaned.sample(5)

,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,page_commentary,page_notes
3350,story,002753,Jade: Keep going.,Oh yeah. How could you forget about his stash ...,1257910481,/storyfiles/hs2/00853_2.gif,[002754],002752,<NA>,[],...,<NA>,<NA>,"[Part 1, Act 3, Jade Harley, Jade's House, Blu...",[Jade],"[[Jade: Proceed through house, Jade: Be trolle...",Rose enters the Hubgrid.\nJade descends throug...,act-3,part-1,<NA>,<NA>
7062,story,005882,[o] ==>,"|PESTERLOG|<br /><span style=""color: #4ac925"">...",1311391422,/storyfiles/hs2/03979.gif,[005883],005881,scratch,[],...,<NA>,<NA>,[],"[Jade, Kanaya, Karkat]","[[Jade: Talk with Karkat, Kanaya, Jade: Rememb...",Scratch leaves.\n\nNoir: Bad dog. Worst enemy....,a5a2,part-2,<NA>,<NA>
1888,story,001583,Next.,Ace Dick's game of LIFE has ended.<br /><br />...,1231387216,/advimgs/ps/ps1370_1.gif,[001584],001582,retro,[],...,<NA>,<NA>,[],[],[],"The Renegade shoots more rockets, then surrend...",act-4,part-1,<NA>,<NA>
6450,story,005356,==>,<NA>,1297128313,/storyfiles/hs2/03453.gif,[005357],005355,<NA>,[],...,<NA>,SCRAAAAAAAAAAAAPE.,[],"[Gamzee, Nepeta]","[[Gamzee: Kill Equius, Gamzee: Engage murder m...",Gamzee cracks Schrödinger’s Joke.,a5a2,part-2,Why is he even doing this? Because murderclown...,<NA>
8428,story,007126,==>,<NA>,1341041015,/storyfiles/hs2/05223.gif,[007127],007125,<NA>,[],...,<NA>,<NA>,[],[Dirk],[[Dirk: Reach edge of dream bubble]],The meteor passes out of Meenah’s bubble. Dirk...,a6-3,part-3,<NA>,<NA>


In [31]:
# Make column types explicit as verification and so they don't resolve to float when we save them, and timestamp stays a timestamp

mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')

# Convert all columns to string except for the lists and timestamp

for column in mspa_df_cleaned.columns:
    if column not in ['timestamp', 'next', 'tags', 'pov_characters', 'pov_commands', 'flag']:
        mspa_df_cleaned[column] = mspa_df_cleaned[column].astype('string')

mspa_df_cleaned.sample(5)

C:\Users\PC\AppData\Local\Temp\ipykernel_21648\4222095517.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')


,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,page_commentary,page_notes
5311,story,004295,Examine caliginous quadrant.,When a pair of adversaries delve into this qua...,2010-08-11 00:07:12,/storyfiles/hs2/02395_1.gif,[004296],004294,<NA>,[],...,<NA>,<NA>,[],[],[],Troll concupiscent romance is explained.,a5a1,part-2,"Okay, I'm back. These are both good examples, ...",<NA>
11166,story,009705,[A6A6I5] ====>,"|DIALOGLOG|<br /><span style=""color: #00d5f2"">...",2015-07-02 22:21:20,/storyfiles/hs2/07801.gif,[009706],<NA>,<NA>,[],...,<NA>,<NA>,[],"[Jane, Nannasprite]","[[Jane: <====, Jane: Talk with Nannasprite, Ja...",Two scenes can be read in any order: Jane meet...,a6a6-5,part-4,<NA>,<NA>
5371,story,004355,Feferi: Disarm.,You decide to unwind and take your mind off th...,2010-08-17 20:04:05,/storyfiles/hs2/02454_1.gif,[004356],004354,<NA>,[],...,<NA>,<NA>,[],[Feferi],"[[Feferi: Drink Tab, Feferi: Disarm.]]",Vriska breaks Eridan’s spade.,a5a1,part-2,<NA>,<NA>
8393,story,007094,==>,"|PESTERLOG|<br /><span style=""color: #323232"">...",2012-06-26 09:04:46,/storyfiles/hs2/05191.gif,[007095],007093,<NA>,[],...,<NA>,<NA>,[],"[Caliborn, Dirk, Lil' Cal]","[[Caliborn: <==, Caliborn: Jeer Calliope about...",Caliborn jeers Calliope about the clusterfuck:...,a6-3,part-3,<NA>,<NA>
5675,story,004645,Terezi: Deploy smelloscope.,"You make use of your trusty SMELLOSCOPE, an it...",2010-10-12 01:16:35,/storyfiles/hs2/02744.gif,[004646],004644,<NA>,[],...,<NA>,<NA>,[],[Terezi],"[[Terezi: Witness destruction of Prospit, Tere...","Previously: from the roof of the lab, Terezi w...",a5a2,part-2,<NA>,<NA>


In [32]:
# Let's reorder the columns to make it easier to read, the first columns should let a user know the story and the title of the page at least

mspa_df_cleaned = mspa_df_cleaned[
    ['viz_story', 'title', 'content', 'page_id', 'timestamp', 'src', 'next', 'previous', 'act', 'part', 'viz_page_number', 'viz_link',
     'page_group_summary', 'tags', 'pov_characters', 'pov_commands', 'transcript_class', 'transcript_style',
     'transcript_tag', 'transcript', 'page_commentary', 'page_notes', 'theme', 'flag']]

mspa_df_cleaned.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
1634,Problem Sleuth,MK: Exit speakeasy.,<NA>,001380,2008-11-21 05:51:17,/advimgs/ps/ps1167.gif,[001381],001379,intermission,part-1,...,[],[],<NA>,<NA>,<NA>,EXIT,<NA>,<NA>,retro,[]
4208,Homestuck,==>,<NA>,003426,2010-03-03 02:55:44,/storyfiles/hs2/01526.gif,[003427],003425,act-4,part-1,...,[Karkat],[[Karkat: # Ask future self why Jade needs to ...,<NA>,<NA>,<NA>,<NA>,Is there anything more useless than the Pester...,<NA>,<NA>,[]
11421,Homestuck,[S] Terezi: Remem8er.,<NA>,009859,2015-07-27 21:03:12,/storyfiles/hs2/07953/07953.swf,[009860],009858,a6a6-5,part-4,...,"[Aradia, Dave, Dirk, Equius, Eridan, Feferi, J...","[[Aradia: Remem8er.], [Dave: Remem8er.], [Dirk...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[S, HQ]"
4363,Homestuck,Future Dream Rose: Cease to exist.,<NA>,003555,2010-04-03 21:54:18,/storyfiles/hs2/01655.gif,[003556],003554,act-4,part-1,...,[Rose],[[Rose: <==]],<NA>,<NA>,<NA>,<NA>,Look how big future Mutini is. What a cute det...,<NA>,<NA>,[]
11651,Other Section,MS Paint Adventures,"<tbody><tr> <td bgcolor=""#FFFFFF""> ...",ps000029,NaT,/extras/ps000029_1.gif,None,<NA>,<NA>,<NA>,...,[],[],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None


In [33]:
# And let's make a dict with column descriptions for later when we release the dataset

dataset_column_descriptions = {
    'viz_story': "The story the page is part of, always 'Homestuck'.",
    'title': "The title of the page, usually the command given in the previous page.",
    'content': "The content of the page, usually the page minus the panels (includes dialog and HTML).",
    'page_id': "The page number, usually a six digit padded string like '001901', sometimes a regular string like 'pony'.",
    'timestamp': "The UNIX timestamp of the page.",
    'src': "The partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like '/storyfiles/hs2/001901.gif'.",
    'next': "The pageId of the next pages, usually a list of one string like ['001902'].",
    'previous': "The pageId of the previous page, usually a string like '001900'.",
    'act': "The act the page is in, always a string like 'act-1'.",
    'part': "The part the page is in, always a string like 'part-1'.",
    'viz_page_number': "Pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ.",
    'viz_link': "The link to the page in the VIZ website.",
    'page_group_summary': "The summary of the group of pages this page is in.",
    'tags': "The tags of the page, like ['John Egbert', 'Rose Lalonde', 'Land of Wind and Shade'].",
    'pov_characters': "The characters that appear in the page according to the POV cam.",
    'pov_commands': "The commands that the characters have in the page according to the POV cam.",
    'transcript_class': "The class of the transcript, like 'dialogue' or 'narration'.",
    'transcript_style': "The style of the transcript, in CSS.",
    'transcript_tag': "The tag of the transcript, like 'p' or 'div'.",
    'transcript': "The transcript of the panel, the text that appears in the panel.",
    'page_commentary': "The commentary of the page, usually by Andrew Hussie.",
    'page_notes': "The notes of the page, usually by volunteers.",
    'theme': "The theme of the page, like 'A6A6'.",
    'flag': "A list of flags for Homestuck Collection presentation."
}

In [34]:
# Before we continue we need to confirm we can fetch the image or asset of each panel
# Pick an asset with a src, like "/storyfiles/hs2/001901.gif"

# We'll use a random homestuck panel with enough metadata, like the one in viz page 2519
panel_src = mspa_df_cleaned[mspa_df_cleaned['viz_page_number'] == '2519']['src'].values[0]
panel_url = f"https://www.homestuck.com/images{panel_src}"

# Test the image URL
print(f"Fetching image from {panel_url}")
requests.get(panel_url)


Fetching image from https://www.homestuck.com/images/storyfiles/hs2/02518.gif


<Response [200]>

In [35]:
Image(url=panel_url)

In [36]:
# Right now we haven't even saved the data, so let's do that first, as csv

mspa_dataset_output_file = os.path.join(OUTPUT_FOLDER, 'mspa_dataset.csv')
mspa_df_cleaned.to_csv(mspa_dataset_output_file, index=False)

# Now we can load the data back (CSV does have the limitation of not representing pd.NA correctly though, we'll just get NaNs)
mspa_df_csv = pd.read_csv(mspa_dataset_output_file)
mspa_df_csv.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4436,Homestuck,==>,NaN,003621,2010-04-22 05:19:18,/storyfiles/hs2/01721.gif,['003622'],3620.0,act-4,part-1,...,"['Dave', ""Lil' Cal"", 'Rose']","[['Dave: Jam with Rose', 'Dave: See Rose throw...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1754,Problem Sleuth,NB: Finish her.,"You fire up your finishing move, LV. 99 CUDDLE...",001481,2008-12-16 06:50:35,/advimgs/ps/ps1268.gif,['001482'],1480.0,intermission,part-1,...,[],[],NaN,NaN,NaN,NaN,"Once you reach your LV. 99 CUDDLETECHS, you ba...",NaN,retro,[]
107,Jailbreak,Put it in the sad recess,You are not deterred by any theoretical discre...,000092,2007-06-04 19:44:31,/advimgs/jb/lv3/mspaintadventure06.gif,['000093'],91.0,act-1,part-1,...,[],[],NaN,NaN,NaN,SQUISH,NaN,NaN,retro,[]
5112,Homestuck,======>,"Why, Aradia. It appears the red glass of your ...",004126,2010-07-14 00:40:41,/storyfiles/hs2/02226.gif,['004127'],4125.0,a5a1,part-2,...,['Equius'],"[['Equius: <======', 'Equius: Talk to Gamzee',...",NaN,NaN,NaN,fg,NaN,NaN,NaN,[]
8299,Homestuck,==>,"You often like to draw your TROLLSONA too, CAL...",007001,2012-06-19 07:46:52,/storyfiles/hs2/05098.gif,['007002'],7000.0,a6-3,part-3,...,['Calliope'],"[['Calliope: Cheer Roxy', 'Calliope: Tell Roxy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


# Augmenting panels with finetuned GPT4-o data

We have the panel images, we have the ReadMSPA transcripts of them, we have the point of view from the POV cam, and we have the title and text that accompanies the panels. We have some partial tagging information from the Homestuck Search engine. With that and a vision model, we might be able to successfully extract non-hallucinated information.
Basically we need to predict the tag-related columns, since we only have tagging data for the first four acts. We'll also need to caption the panels for general LORAs.

In [37]:
# First we need to get only the image we need. We can assemble the image URLs from the src column, by prefixing "https://homestuck.com/images" to it
# That, plus the rest of informational columns, will be useful information to train the model with
# We want to predict the "tags" column given the rest, so we take the Act 1-4 data (basically everything that has a non-empty tags column):

four_acts_hs_df = mspa_df_cleaned[mspa_df_cleaned['tags'].apply(len) > 0]

# There's actually a single tagged act 6 panel (???? HSSE people?) so let's remove everything after viz page 1989
four_acts_hs_df = four_acts_hs_df[four_acts_hs_df['viz_page_number'].astype(int) <= 1989]

four_acts_hs_df.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
2483,Homestuck,John: Put down razor.,Put it...<br /><br />Down?<br /><br />...<br /...,002020,2009-05-06 07:47:20,/storyfiles/hs2/00120.gif,[002021],002019,act-1,part-1,...,[John],"[[John: Be introduced, John: Mess around]]",<NA>,<NA>,<NA>,<NA>,RE: Put it... Down? The narrative is officiall...,<NA>,<NA>,[]
4103,Homestuck,==>,Once you're done you'll captchalogue the bottl...,003341,2010-02-20 17:50:52,/storyfiles/hs2/01441_2.gif,[003342],003340,act-4,part-1,...,[Dave],[[Dave: Talk to dream Jade]],<NA>,<NA>,<NA>,→ SOME CODE → PESTERCHUM hey dude here is a ...,<NA>,<NA>,<NA>,[R4]
4402,Homestuck,==>,<NA>,003590,2010-04-17 05:55:09,/storyfiles/hs2/01690.gif,[003591],003589,act-4,part-1,...,"[John, Rose]","[[John: Be trolled by Terezi, John: Proceed to...",<NA>,<NA>,<NA>,ROSE: Z,This is the first time a Breath player riding ...,<NA>,<NA>,[]
2921,Homestuck,John: Ride Slimer pogo and one-up that imp.,"Well ok, it's not a Slimer pogo, but you mount...",002395,2009-08-18 03:33:46,/storyfiles/hs2/00495.gif,[002396],002394,act-2,part-1,...,"[John, Rose]","[[John: Hammer your way to victory, John: Wond...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
3119,Homestuck,==>,<NA>,002558,2009-09-19 03:47:08,/storyfiles/hs2/00658_1.gif,[002559],002557,act-2,part-1,...,[John],"[[John: Wake up, John: Prepare for boss battle]]",<NA>,<NA>,<NA>,<NA>,John: Steal ogre's huge silly elf shoe and rid...,<NA>,<NA>,[]


In [38]:
# For the dataset we'll need to make a new dataframe with only the relevant data
# We don't need viz_story, timestamp, next, previous, viz_page_number, viz_link
# We'll keep the rest, and remove the tag later when it's time

tagging_dataset = four_acts_hs_df.drop(
    columns=['viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])

tagging_dataset.sample(5)

,title,content,page_id,src,act,part,page_group_summary,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4545,Rose: Combine needlewands and grimoire.,You make the THORNS OF OGLOGOTH.<br /><br />Th...,003712,/storyfiles/hs2/01812_2.gif,act-4,part-1,Rose alchemizes sweet loot.,"[Part 1, Act 4, Rose Lalonde, Dress, Velvet Sq...",[Rose],"[[Rose: Alchemize, Rose: Create Thorns of Oglo...",<NA>,<NA>,<NA>,<NA>,Rose is not an especially sane adventurer. So ...,<NA>,<NA>,[]
3230,WV: Adjust time dial to appearify rotten pumpkin.,You and SERENITY consider new ways to waste mo...,002651,/storyfiles/hs2/00751_2.gif,act-2,part-1,The Vagabond releases Serenity from the amber ...,"[Part 1, Act 2, WV, Exile, Carapacian, Dersite...",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]",<NA>,<NA>,<NA>,THE PAST,Remember how the pumpkin disappeared in one of...,<NA>,<NA>,[]
2465,John: Retreat upstairs!,You pause at the juncture and head down the ha...,002004,/storyfiles/hs2/00104_2.gif,act-1,part-1,John makes a mess in the bathroom.,"[Part 1, Act 1, John's House, John Egbert, Gre...",[John],"[[John: Be introduced, John: Mess around]]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
2737,Dave: Answer chum.,"|PESTERLOG|<br /><span style=""color: #b536da"">...",002233,/storyfiles/hs2/00333.gif,act-2,part-1,Dave browses the web. Rose pesters him about h...,"[Act 2, Part 1, Intro, Dave Strider, Dave's Ro...","[Dave, Rose]","[[Dave: Talk with Rose about playing Sburb], [...",<NA>,<NA>,<NA>,<NA>,I think certain elements within the readership...,<NA>,<NA>,[]
3049,==>,"While you're at it, you dump the contents of t...",002504,/storyfiles/hs2/00604.gif,act-2,part-1,Dave wraps up his kitchen activities.,"[Part 1, Act 2, Puppet, Blood]",[Dave],[[Dave: Puppets]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [39]:
# Art styles are tagged inconsistently in HSSE. Everything that does not have a style tag has the style "Sprite Mode". We'll add that.
# The existing styles we need to search for are "Hero Mode", "Lined Mode" (semisprite), "Homosuck" (homosuck mode), "Hussnasty" (detailed style that is unused in the dataset as of yet), and "Scribble Mode"
# First let's check the tags arrays for the styles, and if it lacks all of them, we'll add "Sprite Mode"

art_styles = ["Sprite Mode", "Hero Mode", "Lined Mode", "Homosuck", "Hussnasty", "Scribble Mode"]

# Use as a df function
def add_sprite_mode_tag_if_other_styles_missing(row: pd.Series) -> pd.Series:
    if not any(style in row['tags'] for style in art_styles):
        row['tags'].append("Sprite Mode")
    return row

tagging_dataset = tagging_dataset.apply(add_sprite_mode_tag_if_other_styles_missing, axis=1)

# Let's remove any file that isn't a png, jpg, jpeg or gif
tagging_dataset = tagging_dataset[tagging_dataset['src'].apply(lambda src: src.endswith(('.png', '.jpg', '.jpeg', '.gif')))]

# Show some data
tagging_dataset.sample(5)

,title,content,page_id,src,act,part,page_group_summary,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4218,==>,<NA>,003436,/storyfiles/hs2/01536_1.gif,act-4,part-1,The Black Queen sends the Mistress to see Jack...,"[Part 1, Act 4, PM, Lined Mode]",[Parcel Mistress],[[Parcel Mistress: Follow AR]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
4613,==>,<NA>,003747,/storyfiles/hs2/01847.gif,act-4,part-1,"Dave copies and reads Rose’s notebooks, then m...","[Part 1, Act 4, Unreal Air, Alchemiter, Juttin...",[Dave],[[Dave: Wake up and jam.]],<NA>,<NA>,<NA>,<NA>,I think it's nice how alternate outfits can be...,<NA>,<NA>,[]
3415,John: Jump down.,<NA>,002810,/storyfiles/hs2/00910.gif,act-3,part-1,John and Jade steel themselves for familial en...,"[Part 1, Act 3, Grist, Roof, Sprite Mode]",[John],"[[John: <==, John: Enter Dad's room]]",<NA>,<NA>,<NA>,⬡ +50,<NA>,<NA>,<NA>,[]
4267,==>,You make a tiny AIR CONDITIONER.<br /><br />Th...,003477,/storyfiles/hs2/01577.gif,act-4,part-1,Dave creates an enlarger upgrade.,"[Part 1, Act 4, Dave Strider, Alchemiter, Spri...","[Dave, Jade]","[[Dave: Perform alchemy, Dave: Upgrade alchemi...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
2901,John: Confront Pogo Ride to prepare yourself f...,Thank God your sanity has returned so you can ...,002375,/storyfiles/hs2/00475_2.gif,act-2,part-1,John assesses the imp situation.,"[Act 2, Part 1, Lined Mode, John Egbert, Green...","[John, Rose]","[[John: Put bunny in strife specibus, John: St...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [40]:
# VERY annoyingly, OpenAI refuses to accept certain images for containing blood
# We can only know this in advance by using the moderation API, so let's do that for all images and descriptions
# the model is omni-moderation-latest, and the example OpenAI gives is 
# response = client.moderations.create(
#     model="omni-moderation-latest",
#     input=[
#         {"type": "text", "text": "...text to classify goes here..."},
#         {
#             "type": "image_url",
#             "image_url": {
#                 "url": "https://example.com/image.png",
#                 # can also use base64 encoded image URLs
#                 # "url": "data:image/jpeg;base64,abcdefg..."
#             }
#         },
#     ],
# )
# We'll just iterate with the lines we were going to give, save the output in a file as we go to prevent us from having to do this again
# We already have the openAI client, so let's go

# Check if the file exists, only start if it doesn't
moderation_output_file = os.path.join(OUTPUT_FOLDER, 'moderation_output.jsonl')

if not os.path.exists(moderation_output_file):
    with open(moderation_output_file, 'w') as f:
        print(f"Moderation output file created at {moderation_output_file}")
        for i, row in tagging_dataset.iterrows():
            print(f"Moderating panel {row['page_id']}...")
            response = openai_client.moderations.create(
                model="omni-moderation-latest",
                input=[
                    # Pass the JSON object
                    {"type": "text", "text": json.dumps(row.drop('tags').to_dict())},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"https://homestuck.com/images{row['src']}"
                        }
                    },
                ],
            )
            openai_response_dict = response.to_dict()
            # We need to add the text and image to the dict, since the response doesn't include them
            openai_response_dict['text'] = row.drop('tags').to_dict()
            openai_response_dict['image_url'] = f"https://homestuck.com/images{row['src']}"
            f.write(json.dumps(openai_response_dict) + '\n')
            print(f"Moderation response for panel {row['page_id']} written to {moderation_output_file}")


In [41]:
# Let's load the moderation output as a dataframe
with open(moderation_output_file, 'r') as f:
    moderation_output = [json.loads(line) for line in f]
    
moderation_output_df = pd.DataFrame(moderation_output)
moderation_output_df.sample(5)

,id,model,results,text,image_url
1465,modr-4f94cfa87263f8f47c216dd9914b9f10,omni-moderation-latest,"[{'categories': {'harassment': False, 'harassm...","{'title': '[I] ==>', 'content': 'You admit the...",https://homestuck.com/images/storyfiles/hs2/01...
1029,modr-218fd4d39ce3bf9df309bb4be762d0ad,omni-moderation-latest,"[{'categories': {'harassment': False, 'harassm...",{'title': 'Jade: Scamper into grand foyer with...,https://homestuck.com/images/storyfiles/hs2/00...
2279,modr-e491d088f996ef87316e01b41b930cb8,omni-moderation-latest,"[{'categories': {'harassment': False, 'harassm...","{'title': 'CG: Troll John.', 'content': '|PEST...",https://homestuck.com/images/storyfiles/hs2/01...
224,modr-f7e4c06fbc94569b484ab513c03a61ac,omni-moderation-latest,"[{'categories': {'harassment': False, 'harassm...","{'title': 'John: High-five Kernelsprite.', 'co...",https://homestuck.com/images/storyfiles/hs2/00...
1477,modr-e58e3335101d0557067a62687af233e2,omni-moderation-latest,"[{'categories': {'harassment': False, 'harassm...","{'title': '[I] SS: Remove knife from eye.', 'c...",https://homestuck.com/images/storyfiles/hs2/01...


In [42]:
# The raw format is a bit annoying, with id (useless) | model (useless) | results (an array of nested dicts) | text (useful for determining reason for moderation) | image_url (same)
# Let's  try to unpack the content of results into something more useful
# [{'categories': {'harassment': False, 'harassment/threatening': False, 'hate': False, 'hate/threatening': False, 'self-harm': False, 'self-harm/instructions': False, 'self-harm/intent': False, 'sexual': False, 'sexual/minors': False, 'violence': True, 'violence/graphic': False, 'illicit': False, 'illicit/violent': False}, 'category_scores': {'harassment': 7.67292412858718e-05, 'harassment/threatening': 3.8596609058077356e-05, 'hate': 4.61127481426412e-06, 'hate/threatening': 1.4510478554719287e-06, 'self-harm': 0.004682315776288989, 'self-harm/instructions': 3.7636447180974406e-06, 'self-harm/intent': 0.00022071847593153172, 'sexual': 0.0007098506750714407, 'sexual/minors': 1.1959857805023158e-05, 'violence': 0.4270457126928944, 'violence/graphic': 0.0024601736421077493, 'illicit': 7.2543618230189156e-06, 'illicit/violent': 5.649793328376294e-06}, 'flagged': True, 'category_applied_input_types': {'harassment': ['text'], 'harassment/threatening': ['text'], 'sexual': ['text', 'image'], 'hate': ['text'], 'hate/threatening': ['text'], 'illicit': ['text'], 'illicit/violent': ['text'], 'self-harm/intent': ['text', 'image'], 'self-harm/instructions': ['text', 'image'], 'self-harm': ['text', 'image'], 'sexual/minors': ['text'], 'violence': ['text', 'image'], 'violence/graphic': ['text', 'image']}}] is an example line
# I'll want to basically get all of these dict/value pairs as their own column so I can sort and analyze them later

# First let's drop the id and model columns
moderation_output_df.drop(columns=['id', 'model'], inplace=True)

# Now let's unpack the results column
moderation_output_df = moderation_output_df.explode('results')

# I want new columns, first for "flagged", then one per category with its true false output, and 'category_applied_input_types' can stay as a single dict column
moderation_output_df['flagged'] = moderation_output_df['results'].apply(lambda result: result['flagged'])
moderation_output_df['categories'] = moderation_output_df['results'].apply(lambda result: result['categories'])
moderation_output_df['category_scores'] = moderation_output_df['results'].apply(lambda result: result['category_scores'])
moderation_output_df['category_applied_input_types'] = moderation_output_df['results'].apply(
    lambda result: result['category_applied_input_types'])

# Now we can drop the results column
moderation_output_df.drop(columns=['results'], inplace=True)

# Show some data
moderation_output_df.sample(5)

,text,image_url,flagged,categories,category_scores,category_applied_input_types
1655,"{'title': '==>', 'content': 'You unlock the th...",https://homestuck.com/images/storyfiles/hs2/01...,False,"{'harassment': False, 'harassment/threatening'...","{'harassment': 0.0007200331438843956, 'harassm...","{'harassment': ['text'], 'harassment/threateni..."
1831,"{'title': '==>', 'content': 'But perhaps an UN...",https://homestuck.com/images/storyfiles/hs2/01...,False,"{'harassment': False, 'harassment/threatening'...","{'harassment': 0.0006290189785644802, 'harassm...","{'harassment': ['text'], 'harassment/threateni..."
866,"{'title': 'Drop pumpkin on Farmstink.', 'conte...",https://homestuck.com/images/storyfiles/hs2/00...,False,"{'harassment': False, 'harassment/threatening'...","{'harassment': 0.04051733594630966, 'harassmen...","{'harassment': ['text'], 'harassment/threateni..."
522,"{'title': '==>', 'content': '|PESTERLOG|<br />...",https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': False, 'harassment/threatening'...","{'harassment': 0.19558252809005924, 'harassmen...","{'harassment': ['text'], 'harassment/threateni..."
2190,{'title': 'Dave: Combine fetus in a jar and se...,https://homestuck.com/images/storyfiles/hs2/01...,False,"{'harassment': False, 'harassment/threatening'...","{'harassment': 0.0006805796002084943, 'harassm...","{'harassment': ['text'], 'harassment/threateni..."


In [43]:
# We need to extract the individual categories and category scores into columns, so that we get "harassment", "harassment_score", "hate", "hate_score", etc.

# First let's get all the categories
categories = set()
for category_dict in moderation_output_df['categories']:
    categories.update(category_dict.keys())
    
categories

{'harassment',
 'harassment/threatening',
 'hate',
 'hate/threatening',
 'illicit',
 'illicit/violent',
 'self-harm',
 'self-harm/instructions',
 'self-harm/intent',
 'sexual',
 'sexual/minors',
 'violence',
 'violence/graphic'}

In [44]:
# Now we can create the columns
for category in categories:
    print(f"Creating columns for {category}")
    moderation_output_df[f"{category}_flag"] = moderation_output_df['categories'].apply(lambda category_dict: category_dict.get(category, False))
    print(f"Creating columns for {category}_score")
    moderation_output_df[f"{category}_score"] = moderation_output_df['category_scores'].apply(lambda category_dict: category_dict.get(category, 0.0))
    
# Drop the original categories and category_scores columns
moderation_output_df.drop(columns=['categories', 'category_scores'], inplace=True)

# Show some data
moderation_output_df.sample(5)

Creating columns for hate/threatening
Creating columns for hate/threatening_score
Creating columns for illicit
Creating columns for illicit_score
Creating columns for illicit/violent
Creating columns for illicit/violent_score
Creating columns for violence/graphic
Creating columns for violence/graphic_score
Creating columns for self-harm
Creating columns for self-harm_score
Creating columns for hate
Creating columns for hate_score
Creating columns for violence
Creating columns for violence_score
Creating columns for self-harm/intent
Creating columns for self-harm/intent_score
Creating columns for harassment
Creating columns for harassment_score
Creating columns for sexual/minors
Creating columns for sexual/minors_score
Creating columns for self-harm/instructions
Creating columns for self-harm/instructions_score
Creating columns for harassment/threatening
Creating columns for harassment/threatening_score
Creating columns for sexual
Creating columns for sexual_score


,text,image_url,flagged,category_applied_input_types,hate/threatening_flag,hate/threatening_score,illicit_flag,illicit_score,illicit/violent_flag,illicit/violent_score,...,harassment_flag,harassment_score,sexual/minors_flag,sexual/minors_score,self-harm/instructions_flag,self-harm/instructions_score,harassment/threatening_flag,harassment/threatening_score,sexual_flag,sexual_score
1153,"{'title': '==>', 'content': None, 'page_id': '...",https://homestuck.com/images/storyfiles/hs2/01...,False,"{'harassment': ['text'], 'harassment/threateni...",False,8.139693e-07,False,0.000021,False,0.000021,...,False,0.000033,False,0.000005,False,0.000002,False,0.000012,False,0.000103
1085,"{'title': '==>', 'content': 'Your blood is boi...",https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': ['text'], 'harassment/threateni...",False,4.757645e-06,False,0.000038,False,0.000018,...,False,0.009169,False,0.000008,False,0.000005,False,0.001191,False,0.000704
1426,{'title': '[I] HB: Retrieve two of hearts from...,https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,1.273110e-05,False,0.005294,False,0.000076,...,False,0.005917,False,0.000014,False,0.000230,False,0.005989,False,0.002764
754,"{'title': 'John: Turn around...', 'content': '...",https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': ['text'], 'harassment/threateni...",False,5.064472e-06,False,0.000033,False,0.000027,...,False,0.005117,False,0.000008,False,0.000002,False,0.000972,False,0.010184
1487,{'title': '[I] SS: Stick Crowbar's pin back in...,https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,7.554056e-05,False,0.168651,False,0.089832,...,False,0.140558,False,0.000012,False,0.000320,False,0.039945,False,0.000091


In [45]:
# Now let's get the top 5 URL/text pairs for each category in a dataframe

top_5_moderated = pd.DataFrame()

for category in categories:
    top_5 = moderation_output_df.sort_values(by=f"{category}_score", ascending=False).head(5)
    top_5['category'] = category
    top_5_moderated = pd.concat([top_5_moderated, top_5])
    
# put the category column first
top_5_moderated = top_5_moderated[['category'] + [col for col in top_5_moderated.columns if col != 'category']]

top_5_moderated

,category,text,image_url,flagged,category_applied_input_types,hate/threatening_flag,hate/threatening_score,illicit_flag,illicit_score,illicit/violent_flag,...,harassment_flag,harassment_score,sexual/minors_flag,sexual/minors_score,self-harm/instructions_flag,self-harm/instructions_score,harassment/threatening_flag,harassment/threatening_score,sexual_flag,sexual_score
663,hate/threatening,{'title': 'Dave: Search for some MAD SNACKS YO...,https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.004274,False,0.114405,False,...,False,0.196002,False,0.000009,False,0.028689,False,0.119407,False,0.000707
1958,hate/threatening,"{'title': 'Davesprite: Troll GC.', 'content': ...",https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.004257,False,0.020036,False,...,False,0.306562,False,0.000468,False,0.010868,False,0.119466,False,0.065673
1268,hate/threatening,"{'title': 'Dave: Answer.', 'content': '|PESTER...",https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.004215,False,0.007531,False,...,False,0.342288,False,0.056841,False,0.000352,False,0.007261,False,0.563770
1522,hate/threatening,"{'title': '[I] ==>', 'content': None, 'page_id...",https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.003569,False,0.169241,False,...,False,0.019544,False,0.000011,False,0.000302,False,0.038554,False,0.000167
1645,hate/threatening,{'title': 'Jade: Deploy the cruxtruder in its ...,https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.003529,False,0.020060,False,...,False,0.056087,False,0.000395,False,0.000229,False,0.040046,False,0.010504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,sexual,"{'title': '==>', 'content': '|PESTERLOG|<br />...",https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.000008,False,0.000006,False,...,False,0.027395,False,0.000276,False,0.000218,False,0.000946,True,0.932177
576,sexual,"{'title': 'Rose: Answer Dave.', 'content': '|P...",https://homestuck.com/images/storyfiles/hs2/00...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.000007,False,0.000007,False,...,False,0.028183,False,0.000388,False,0.000217,False,0.000974,True,0.931142
1255,sexual,"{'title': 'Dave: Pester Rose.', 'content': '|P...",https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.000005,False,0.000006,False,...,False,0.005638,False,0.000105,False,0.000002,False,0.000761,True,0.780469
1464,sexual,{'title': '[I] CD: Burst in thrusting bull pen...,https://homestuck.com/images/storyfiles/hs2/01...,True,"{'harassment': ['text'], 'harassment/threateni...",False,0.000023,False,0.000034,False,...,False,0.107821,False,0.000080,False,0.000225,False,0.006215,False,0.571190


In [46]:
# Try to get the most positive panels, the ones that rate the lowest in all categories and have flagged=False
# First let's sum all the scores into one regular "flagged_score" column and sort by that

moderation_output_df['flagged_score'] = moderation_output_df[[f"{category}_score" for category in categories]].sum(axis=1)
least_flagged = moderation_output_df[~moderation_output_df['flagged']].sort_values(by='flagged_score', ascending=True)

# keep the columns that matter to us (the flagged_score, text and image_url)
least_flagged = least_flagged[['flagged', 'flagged_score', 'text', 'image_url']]

# Show some data, sorted
least_flagged.head(5)

,flagged,flagged_score,text,image_url
1692,False,0.000000,"{'title': '==>', 'content': '|PESTERLOG|<br />...",https://homestuck.com/images/storyfiles/hs2/01...
1230,False,0.000042,{'title': 'John: Combine Dad's hat and Problem...,https://homestuck.com/images/storyfiles/hs2/01...
1671,False,0.000050,{'title': 'Hooray! This one contains a prize!'...,https://homestuck.com/images/storyfiles/hs2/01...
132,False,0.000050,"{'title': 'John: Get free Fetch Modus.', 'cont...",https://homestuck.com/images/storyfiles/hs2/00...
905,False,0.000052,{'title': 'Jade: Change wardrobifier setting.'...,https://homestuck.com/images/storyfiles/hs2/00...


In [47]:
# Get the text for image_url https://homestuck.com/images/storyfiles/hs2/01066_1.gif since it seems to be a relatively wholesome panel
# We'll use the text in the system prompt for the tagger

system_example = least_flagged[least_flagged['image_url'] == 'https://homestuck.com/images/storyfiles/hs2/01066_2.gif']['text'].values[0]
system_example

{'title': "John: Combine Dad's hat and Problem Sleuth game.",
 'content': 'You make another ordinary FEDORA with FOUR PIECES OF CANDY CORN inside.',
 'page_id': '002966',
 'src': '/storyfiles/hs2/01066_2.gif',
 'act': 'act-3',
 'part': 'part-1',
 'page_group_summary': 'John alchemizes so much sweet loot.',
 'pov_characters': ['John'],
 'pov_commands': [['John: Alchemise']],
 'transcript_class': None,
 'transcript_style': None,
 'transcript_tag': None,
 'transcript': None,
 'page_commentary': None,
 'page_notes': None,
 'theme': None,
 'flag': []}

In [64]:
"""
Now let's turn the tagging dataset into OpenAI's JSONL format, an example of which is here:

```jsonl
{
  "messages": [
    { "role": "system", "content": "You are an assistant that identifies uncommon cheeses." },
    { "role": "user", "content": "What is this cheese?" },
    { "role": "user", "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg"
          }
        }
      ]
    },
    { "role": "assistant", "content": "Danbo" }
  ]
}
```

Each row of the dataset will turn into something here. 
The system prompt will contain an example of the input data, so let's deal with that later.
The first user content will be the metadata minus the image and the tags.
The second user content will be the image URL, which we can once again get from https://homestuck.com/images plus the src column. No need to base64 encode it.
The assistant content will be the tags column.

Note: this is only for tagging, but we'll end up reusing this model for captioning too, because it got seriously expensive (90 dollars!) without me realizing.
Luckily it looks like GPT-4o is smart enough to adapt to a different system prompt.
"""

system_prompt = f"""You are a professional image tagger.
Your current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.
You will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.
The image alongside will be the panel itself. Here's an example of the metadata input:
```
{system_example}
```
Note the pov information applies to the page, not to all the panels in it, and a character can not appear in the panel and simply be part of an online conversation within the content, so it's not a perfect predictor of the character-related tags for the specific panel being tagged, but it can help you discriminate if you don't know the characters. A panel always has a style tag, Sprite Mode being a simplistic style with few details, Hero Mode being a more detailed style, Lined Mode being an inbetween style of the previous two with wider strokes, and Scribble Mode being a rough style.

This is an example of your tagged output, which must be a valid JSON list:
```
["John Egbert", "Wise Guy Slime Suit", "Sprite Mode"]
```
"""

def create_openai_jsonl_entry(row: pd.Series) -> dict:
    """
    Creates an OpenAI JSONL entry from a row of the tagging dataset.

    Args:
        row (pd.Series): A row of the tagging dataset.

    Returns:
        dict: An OpenAI JSONL entry.
    """
    # We must remove the organizational tags like "Act 1" and "Part 1" from the tags, because
    # the model cannot predict them
    usable_tags = [tag for tag in row['tags'] if "Act " not in tag and "Part " not in tag]
    # Tags will also be stringified so the model can predict them
    usable_tags = json.dumps(usable_tags)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": json.dumps(row.drop('tags').to_dict())},
        {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"https://homestuck.com/images{row['src']}"
                }
            }
        ]},
        {"role": "assistant", "content": usable_tags}
    ]
    return {"messages": messages}


# Test it with one row
create_openai_jsonl_entry(tagging_dataset.iloc[3])

{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{\'title\': "John: Combine Dad\'s hat and Problem Sleuth game.", \'content\': \'You make another ordinary FEDORA with FOUR PIECES OF CANDY CORN inside.\', \'page_id\': \'002966\', \'src\': \'/storyfiles/hs2/01066_2.gif\', \'act\': \'act-3\', \'part\': \'part-1\', \'page_group_summary\': \'John alchemizes so much sweet loot.\', \'pov_characters\': [\'John\'], \'pov_commands\': [[\'John: Alchemise\']], \'transcript_class\': None, \'transcript_s

In [49]:
# Okay, that is stupid, but let's extract all the image_urls that got flagged = True, and also the ratios

flagged_images = moderation_output_df[moderation_output_df['flagged']]
print(f"Flagged images: {len(flagged_images)} / {len(moderation_output_df)}")

# Create a copy of tagging_dataset without the flagged panels
print("Tagging dataset before removing flagged images:", len(tagging_dataset))
clean_tagging_dataset = tagging_dataset[~tagging_dataset['src'].apply(lambda src: f"https://homestuck.com/images{src}" in flagged_images['image_url'].values)]
print("Tagging dataset after removing flagged images:", len(clean_tagging_dataset))

# Remove some additional flagged panels that only fail when finetuning (???)
srcs_to_remove = [
    '/storyfiles/hs2/01232_2.gif',
    '/storyfiles/hs2/01231.gif',
]

clean_tagging_dataset = clean_tagging_dataset[~clean_tagging_dataset['src'].isin(srcs_to_remove)]

# Show some data
clean_tagging_dataset.sample(5)

Flagged images: 697 / 2397
Tagging dataset before removing flagged images: 2397
Tagging dataset after removing flagged images: 1700


,title,content,page_id,src,act,part,page_group_summary,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4459,==>,<NA>,003634,/storyfiles/hs2/01734.gif,act-4,part-1,Dream Rose tosses Lil Cal out of Dave’s tower....,"[Part 1, Act 4, Scenery, LOWAS, Dad Egbert, Da...","[Dad, Grandpa]","[[Dad: Receive hat, Dad: Meet Grandpa, Dad: Te...",<NA>,<NA>,<NA>,<NA>,"All right, I'll give up the game on something....",<NA>,<NA>,[]
3286,Jade: Change wardrobifier setting.,You deactivate the WARDROBIFIER'S randomizatio...,002700,/storyfiles/hs2/00800_1.gif,act-3,part-1,Jade plays with dolls and admires fauna.,"[Part 1, Act 3, Wardrobifier, Light Symbol, Be...",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
4658,==>,He is prepared for the occasion with a small p...,003779,/storyfiles/hs2/01879.gif,act-4,part-1,Using a terminal calibrated by recent visitors...,"[Part 1, Act 4, Screen, Earth, Scenery, City, ...","[Bro, Dave, John, Rose]","[[Bro: ==>, Bro: ~ Dirk, Bro: Find and adopt D...",<NA>,<NA>,<NA>,<NA>,"Here's Bro, ready with a tiny pair of shades. ...",<NA>,<NA>,[]
4744,Hours in the future...,"The warweary calls another broken planet home,...",003843,/storyfiles/hs2/01943_2.gif,act-4,part-1,The Villein rends the effigy of Jack and later...,"[Part 1, Act 4, WV, Future Earth, Exile, Carap...",[Warweary Villein],[[Warweary Villein: Crash battleship]],<NA>,<NA>,<NA>,<NA>,These swiftly narrated inter-Act panels get us...,<NA>,<NA>,[]
4233,==>,"Ticket? Oh, this thing. Ha, ha, look at that, ...",003451,/storyfiles/hs2/01551.gif,act-4,part-1,Jack offers the green box in exchange for Pros...,"[Part 1, Act 4, PM, Derse, Lined Mode]","[Jack Noir, Parcel Mistress]","[[Jack Noir: PM getting box], [Parcel Mistress...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [50]:
# OpenAI's platform lets you upload two jsonl files, one with the training data and another with the validation data
# We'll split the dataset into 80% training and 20% validation, but we need to shuffle it first

shuffled_tagging_dataset = clean_tagging_dataset.sample(frac=1, random_state=42)
training_size = int(len(clean_tagging_dataset) * 0.8)
training_data = shuffled_tagging_dataset.iloc[:training_size]
validation_data = shuffled_tagging_dataset.iloc[training_size:]

# Remove the training and validation files if they don't already exist
# Remember we're storing them in output, as "hs_tagging_tranining_data.jsonl" and "hs_tagging_validation_data.jsonl"

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Remove the files if they exist
training_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_training_data.jsonl")
validation_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_validation_data.jsonl")

if os.path.exists(training_file):
    os.remove(training_file)
if os.path.exists(validation_file):
    os.remove(validation_file)

# Create the training and validation jsonl files
with open(training_file, 'w') as training_jsonl, open(validation_file, 'w') as validation_jsonl:
    for _, row in training_data.iterrows():
        training_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Training data written to {training_file}")
    for _, row in validation_data.iterrows():
        validation_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Validation data written to {validation_file}")

# Let's load the first line as a json
with open(training_file, 'r') as f:
    first_line_json = json.loads(f.readline())
    
first_line_json

Training data written to ../output\hs_tagging_training_data.jsonl
Validation data written to ../output\hs_tagging_validation_data.jsonl


{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{\'title\': "John: Combine Dad\'s hat and Problem Sleuth game.", \'content\': \'You make another ordinary FEDORA with FOUR PIECES OF CANDY CORN inside.\', \'page_id\': \'002966\', \'src\': \'/storyfiles/hs2/01066_2.gif\', \'act\': \'act-3\', \'part\': \'part-1\', \'page_group_summary\': \'John alchemizes so much sweet loot.\', \'pov_characters\': [\'John\'], \'pov_commands\': [[\'John: Alchemise\']], \'transcript_class\': None, \'transcript_s

In [51]:
# The new model is ft:gpt-4o-2024-08-06:record-crash:hs-tagger:AFSH9n1Q
# Let's try it out with one of the entries not in the training data
# like something from act 5, using mspa_df_cleaned and viz page 2667

# Get the panel
a5a2_panel = mspa_df_cleaned[mspa_df_cleaned['viz_page_number'] == '2667']
# Turn it into a JSONL entry and then call the model, we'll need to drop these colums too:
# columns=['tags', 'viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])
a5a2_panel_x = a5a2_panel.drop(columns=['tags', 'viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])

a5a2_panel_x

,title,content,page_id,src,act,part,page_group_summary,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
5598,John: Dispatch these pests.,"That is IT. <br /><br /><span style=""font-styl...",004567,/storyfiles/hs2/02666.gif,a5a2,part-2,Vriska trolls John about his ransacked block. ...,"[John, Vriska]","[[John: Continue talking with Vriska, John: Us...",<NA>,<NA>,<NA>,<NA>,John telling all the sad imps to GTFO is proba...,<NA>,<NA>,[]


In [52]:
TAGGING_MODEL = "ft:gpt-4o-2024-08-06:record-crash:hs-tagger:AFSH9n1Q"
# Call the model
def prompt_openai_tagging_model(row: pd.Series) -> dict:
    image_url = f"https://homestuck.com/images{row['src']}"
    completion = openai_client.chat.completions.create(
        model=TAGGING_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(row.to_dict())},
            {"role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ]}
        ]
    )
    return completion.choices[0].message
    
    
# Test it
tagging_response = prompt_openai_tagging_model(a5a2_panel_x.iloc[0])

tagging_response


ChatCompletionMessage(content='["John Egbert", "Wise Guy Slime Suit", "Imp", "Sprite Mode"]', role='assistant', function_call=None, tool_calls=None, refusal=None)

In [75]:
"""To save money we'll use OpenAI's batched requests API, where we save the requests to a file 
and upload it for it to be processed asynchronously
to quote: Batches start with a .jsonl file where each line contains the details of an individual request to the API. 
example:
```jsonl
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
```
We have one working request above for an arbitrary panel, we'll go for every untagged homestuck panel, that is, everything that wasn't in four_acts_hs_df = mspa_df_cleaned[mspa_df_cleaned['tags'].apply(len) > 0]
We can invert the condition to get the untagged panels, and then we can use the prompt_openai_tagging_model function to get the responses.

Updated note: it was going to be 20 dollars even with just the batch API to tag the panels. Since we're already passing this to the 90 dollar model, let's make it also caption the panels at the same time, no sense doing this twice. I'll make it describe the scene first, even, so the tags are enforced to make sense.
"""

# Get the untagged panels, not just by searching for tags but also vizStory == Homestuck
untagged_panels = mspa_df_cleaned[(mspa_df_cleaned['tags'].apply(len) == 0) & (mspa_df_cleaned['viz_story'] == 'Homestuck')]

# No need to remove flagged panels since none are in the above dataset
# Let's save the requests to a file
FORCE_TAGGING = False
untagged_panels_file = os.path.join(OUTPUT_FOLDER, 'untagged_panels.jsonl')
        
def create_panel_batch_request(panels_to_tag, tagging_system_prompt: str) -> str:
    # 10 requests cost $0.03, the total cost will be $0.03 * (len(untagged_panels) / 10)
    print(f"Expected cost for tagging: ${0.03 * (len(panels_to_tag) / 10)}") # 20 dollars
    with open(untagged_panels_file, 'w') as f:
        for _, row in panels_to_tag.iterrows():
            print(f"Writing request for panel {row['src']}...")
            # We don't drop tags since the act1-4 tags might be useful for the captioner
            row_serializable = row.drop(['viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])
            # I wish we could grab act summary for the panel but it adds so many tokens, maybe when the models are cheaper
            # row_serializable['act_summary'] = act_summaries[row['act']]
            f.write(json.dumps({
                "custom_id": row_serializable['src'],
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": TAGGING_MODEL,
                    "messages": [
                        {"role": "system", "content": tagging_system_prompt},
                        {"role": "user", "content": json.dumps(row_serializable.to_dict())},
                        {"role": "user", "content": [
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"https://homestuck.com/images{row['src']}"
                                }
                            }
                        ]}
                    ]
                }
            }) + '\n')
        print(f"Untagged panels requests written to {untagged_panels_file}")
    return untagged_panels_file


In [76]:
"""
TODO: update when we actually have the complete dataset (i.e. tagged Homestuck act 5 onwards)
For now we'll go with four_acts_hs_df even though that's missing Hussnasty style and a bunch of characters

We can make LORAs for styles and characters since our content is almost perfectly tagged. However, LORAs are trained with natural language strings that use unusual strings. Instead of John Egbert, it probably wants something that never appeared in its datasets like hsjohnegbert or something. It also needs to be part of a sentence, like `a trtcrd of a woman wearing a crown of stars, sitting on a throne, holding a scepter, dressed in a robe with a pomegranate pattern, with a field and trees in the background, "the empress"`. 

These natural descriptions will require using GPT-4o once again, but this time with a LORA prompt, and then using the output of image/description pairs to train the LORA model.

Additionally, the special strings will need to be hardcoded, and have the model use them whenever the related tag appears and we're in the right dataset... if we're training a John Egbert model, we need to tell the model to use the special string in its description.

(Note: https://huggingface.co/datasets/multimodalart/1920-raider-waite-tarot-public-domain, prioritize flux training?)
(https://replicate.com/ostris/flux-dev-lora-trainer/train for training styles, https://fal.ai/models/fal-ai/flux-lora-fast-training for training characters) 
"""

caption_example = {"caption": "A Sprite Mode panel of John Egbert wearing his Wise Guy Slime Suit standing next to a hat on the floor", "tags": ["John Egbert", "Wise Guy Slime Suit", "Sprite Mode"]}

lora_system_prompt = f"""You are a professional image captioner and tagger.
Your current project is creating captions and character/location/unique object tags for the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.
You will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character POVs and summaries of page groups.
The image alongside will be the panel itself. Here's an example of the metadata input:
```
{system_example}
```
Note the pov information applies to the page, not to all the panels in it, and a tagged/pov character can not appear in the panel and simply be part of an online conversation within the content. A panel almost always has a style tag, Sprite Mode being a simplistic style with few details, Hero Mode being a more detailed style, Lined Mode being an inbetween style of the previous two with wider strokes, and Scribble Mode being a rough style. Default to Sprite Mode if the style is inconclusive.

This is an example of your captioned output, which must be a valid JSON object representing a visual description of the image as redacted for the LORA model, and an array of tags:
```
{caption_example}
```
"""

# Trying out OpenAI's free prompt generator
openai_lora_system_prompt = """You are a professional image captioner and tagger. Your task is to create detailed captions and tag character/location/unique object features for panels of the webcomic Homestuck. You will receive metadata for each panel, including transcripts of text, dialogue, narration, character POVs, and summaries. Use this information, alongside the image itself, to generate accurate and comprehensive captions and tags.

# Steps

1. **Analyze the Metadata:** 
   - Review information such as the title, content, act, part, page group summary, and character POVs.
   - Understand that POV information applies to the page, not all panels, and characters may be part of online conversations without appearing in the panel.

2. **Determine Style Tag:**
   - Identify the style of the panel: Sprite Mode, Hero Mode, Lined Mode, or Scribble Mode.
   - Use Sprite Mode as the default if the style is inconclusive.

3. **Create the Caption:**  
   - Formulate a visual description of the image based on metadata, taking care to describe interactions, objects, and relevant visual details.
   
4. **Generate Tags:**
   - Identify and list tags for characters, locations, and distinctive objects or actions present in the panel.
   - Human-provided tags might occassionally be included in the given metadata, in which case you can just re-use them.

5. **Compile into JSON:**  
   - Structure the output as a valid JSON object with two primary fields: 'caption' and 'tags'.

# Output Format

The output should be a JSON object with:
- `caption`: A string describing the visual details of the image.
- `tags`: An array of relevant tags encompassing characters, locations, objects, and styles.

# Examples

Input metadata:

```json
{
  'title': "John: Combine Dad's hat and Problem Sleuth game.",
  'content': 'You make another ordinary FEDORA with FOUR PIECES OF CANDY CORN inside.',
  'page_id': '002966',
  'src': '/storyfiles/hs2/01066_2.gif',
  'act': 'act-3',
  'part': 'part-1',
  'page_group_summary': 'John alchemizes so much sweet loot.',
  'pov_characters': ['John'],
  'pov_commands': [['John: Alchemise']],
  'transcript_class': None,
  'transcript_style': None,
  'transcript_tag': None,
  'transcript': None,
  'page_commentary': None,
  'page_notes': None,
  'theme': None,
  'flag': [],
  'tags': [],
}
```

Expected output:

```json
{
  "caption": "John is seen combining his Dad's hat with the Problem Sleuth game, resulting in a regular Fedora with four candy corns inside.",
  "tags": ["John", "Dad's Hat", "Problem Sleuth Game", "Fedora", "Candy Corn", "Sprite Mode"]
}
```

# Notes

- Ensure accuracy and relevance in tags and descriptions for clarity and context.
- Accurate style determination is crucial for consistency in tagging."""


# Call the function with the system prompt, and not just the untagged panels but all the panels
# that are in the correct formats
all_homestuck_image_panels = mspa_df_cleaned[mspa_df_cleaned['viz_story'] == 'Homestuck']
# filter the correct filenames
all_homestuck_image_panels = all_homestuck_image_panels[all_homestuck_image_panels['src'].apply(lambda src: src.endswith(('.png', '.jpg', '.jpeg', '.gif')))]

if not os.path.exists(untagged_panels_file) or FORCE_TAGGING:
    create_panel_batch_request(all_homestuck_image_panels, openai_lora_system_prompt)

Expected cost for tagging: $26.592
Writing request for panel /storyfiles/hs2/00002.gif...
Writing request for panel /storyfiles/hs2/00001.gif...
Writing request for panel /storyfiles/hs2/00004.gif...
Writing request for panel /storyfiles/hs2/00003.gif...
Writing request for panel /storyfiles/hs2/00008.gif...
Writing request for panel /storyfiles/hs2/00005.gif...
Writing request for panel /storyfiles/hs2/00009.gif...
Writing request for panel /storyfiles/hs2/00007_1.gif...
Writing request for panel /storyfiles/hs2/00007_2.gif...
Writing request for panel /storyfiles/hs2/00006.gif...
Writing request for panel /storyfiles/hs2/00012.gif...
Writing request for panel /storyfiles/hs2/00011.gif...
Writing request for panel /storyfiles/hs2/00010.gif...
Writing request for panel /storyfiles/hs2/00014.gif...
Writing request for panel /storyfiles/hs2/00016.gif...
Writing request for panel /storyfiles/hs2/00015.gif...
Writing request for panel /storyfiles/hs2/00017.gif...
Writing request for panel 

In [56]:
"""
Let's examine the tags the dataset actually uses. We can check the json file in the HSSE_TAGS constant for a hierarchy of tags. As a reminder, the file is organized like this:

```json
{
  "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
...
```

Painfully, we can't actually determine which tags refer to a "final character". Rose has a child tag, "Rosesprite", so we can't rely on leaf elements.
We'll just have to play it by ear. We can start by getting all the tags that are children of "Character" and have been used more than 30 times.
"""

# Load the tags data
with open(HSSE_TAGS, 'r', encoding='utf-8') as f:
    tags_data = json.load(f)

# Function to get all nodes under a tag
def get_all_tag_members(tag_name: str) -> list:
    # Get the tag id from the tag name
    tag_id = [tag['name'] for tag in tags_data['definitions'].values()].index(tag_name)

    # Get the tag data
    tag = tags_data['definitions'][str(tag_id)]

    # Get the children of the tag
    children = tag['children']

    # If the tag has no children, return the tag name
    if not children:
        return [tag['name']]

    # Otherwise, get the members of the children
    members = [tag['name']]
    for child_id in children:
        members.extend(get_all_tag_members(tags_data['definitions'][str(child_id)]['name']))

    return members

character_tags = get_all_tag_members("Character")

# This is very annoying
organizational_tags = ["Character", "Human", "Kid", "Beta Kid", "Alpha Kid", "Guardian", "Alpha Guardian", "Real Person", "Troll", "Beta Troll", "Alpha Troll", "Cherub", "Felt", "Carapacian", "Exile", "Denizen", "Sprite", "Lusus", "Cat", "Horse", "Dog", "Frog", "Crow", "Bird", "Firefly", "Animal", "Insect", "Creature", "Angel", "Underling", "Drone", "Robot", "First Guardian", "Consort", "The Felt", "Exile", "Ancestor", "SBaHJ", "SbaHJ Drawing", "SBaHJifier", "Dersite"]

# Remove the organizational tags
character_tags = [tag for tag in character_tags if tag not in organizational_tags]
character_tags

['John Egbert',
 'Rose Lalonde',
 'Rosesprite',
 'Dave Strider',
 'Davesprite',
 'Jade Harley',
 'Jane Crocker',
 'Roxy Lalonde',
 'Dirk Strider',
 'Jake English',
 'Beta Guardian',
 'Dad Egbert',
 'Beta Bro',
 'Beta Mom',
 'Grandpa',
 'Nanna',
 'Nannasprite',
 'Dad Crocker',
 'Alpha Bro',
 'Alpha Mom',
 'Grandma',
 'Poppop',
 'Dad',
 'Dad Egbert',
 'Dad Crocker',
 'Bro',
 'Beta Bro',
 'Alpha Bro',
 'Mom',
 'Beta Mom',
 'Alpha Mom',
 'Liv Tyler',
 'Charles Dutton',
 'Charles Barkley',
 'Bill Cosby',
 'Bing Crosby',
 'Eddie Morton',
 'Nick Offerman',
 'Nicolas Cage',
 'Matthew McConaughey',
 'Snoop Dogg',
 'Mr. T',
 'Mr. T Puppet',
 'Foam Fetal Mr. T in a Jar',
 'Will Smith',
 'ICP',
 'Violent J',
 'Shaggy 2 Dope',
 'Guy Fieri',
 'Nick Cage',
 'Owen Wilson',
 'Ben Stiller',
 'Donald Glover',
 'Andrew Hussie',
 'Sweet Bro',
 'Hella Jeff',
 'Geromy',
 'SBaHJ Plush',
 'Karkat Vantas',
 'Aradia Megido',
 'Aradiasprite',
 'Aradiabot',
 'Tavros Nitram',
 'Tavrosprite',
 'GCatavrosprite',
 'So

In [57]:
# Works like a charm, but we need another function. Some character tags might have never been used, so let's extract all the rows that have a character tag and get all the unique characters 
# To do this efficiently, we'll first use pandas to extract all tags, combine all "tags" arrays into one that we can then parse

# Get all tags, sorted by number of appearances by exploding them all and then counting them
all_tags = four_acts_hs_df['tags'].explode().value_counts()

# Drop organizational/style ones
useful_tags = all_tags[~all_tags.index.str.contains("Part |Act | Mode")]

useful_tags


tags
John Egbert                516
John's House               400
Carapacian                 340
Green Slime Ghost Shirt    287
Computer                   282
                          ... 
Swedish Fish                 1
Gummy Bears                  1
Cancer Shirt                 1
TV Antenna                   1
Watermark                    1
Name: count, Length: 555, dtype: int64

In [58]:
# Get all the character tags in the dataset if their count is over 30
character_tags_in_dataset = useful_tags[useful_tags.index.isin(character_tags) & (useful_tags > 30)].index.tolist()

# Remove repeats
character_tags_in_dataset = list(set(character_tags_in_dataset))

# Show some character tags
character_tags_in_dataset

['Stitch',
 'Dad',
 'Harlequinsprite',
 'WV',
 'Spades Slick',
 'HB',
 'Jack Noir',
 'Hearts Boxcars',
 'Serenity',
 'CD',
 'Clubs Deuce',
 'Diamonds Droog',
 'Grandpa',
 'Dad Egbert',
 'Doze',
 'Jade Harley',
 'Eggs',
 'Vodka Mutini',
 'Midnight Crew',
 'Dave Strider',
 'Clover',
 'Crowbar',
 'Rose Lalonde',
 'Cans',
 'Sawbuck',
 'Nanna',
 'John Egbert',
 'DD',
 'Itchy',
 'PM',
 'Jaspers',
 'Ogre',
 'AR',
 'Imp']

In [59]:
# From the given list, let's just keep the relevant ones I know we have enough data for:
# John Egbert, Rose Lalond, Dave Strider, Jade Harley, Dad, Spades Slick, Jack Noir, DD, Grandpa, PM, WV, Imp, Ogre 

CHARACTER_LORA_TAGS = ["John Egbert", "Rose Lalond", "Dave Strider", "Jade Harley", "Dad", "Spades Slick", "Jack Noir", "DD", "Grandpa", "PM", "WV", "Imp", "Ogre"]


In [60]:
# Now the function to create a LORA dataset ("image", "caption") for a specific character tag, then we'll just loop. 
# We'll use a "normalized" filename based on the tag name, like "johnegbert" for "John Egbert". 
# We'll assume that the given dataframe has a "caption" column that is the text description of the image, and a "src" column that contains a partial URL
# we can turn that into a full URL by prefixing "https://homestuck.com/images" to it as always
# Note that the final dataset will be csv with two columns, "image" and "caption

def create_lora_dataset_for_character_tag(df: pd.DataFrame, character_tag: str) -> pd.DataFrame:
    """
    Creates a LORA dataset for a specific character tag.

    Args:
        df (pd.DataFrame): The dataset to create the LORA dataset from.
        character_tag (str): The character tag to create the LORA dataset for.

    Returns:
        pd.DataFrame: The LORA dataset for the character tag.
    """
    # Get the normalized character tag
    normalized_character_tag = character_tag.lower().replace(" ", "")

    # Filter the dataset for the character tag
    character_df = df[df['tags'].apply(lambda tags: character_tag in tags)]
    
    # Filter for png / jpg / jpeg / gif files
    character_df = character_df[character_df['src'].apply(lambda src: src.endswith(('.png', '.jpg', '.jpeg', '.gif')))]

    # Create the LORA dataset
    lora_dataset = character_df[['src', 'content']].copy()
    lora_dataset['src'] = lora_dataset['src'].apply(lambda src: f"https://homestuck.com/images{src}")
    # TODO: replace "content" with an actual caption column later
    lora_dataset.rename(columns={'src': 'image', 'content': 'caption'}, inplace=True)

    # Save it as a csv in OUTPUT_FOLDER
    lora_dataset_output_file = os.path.join(OUTPUT_FOLDER, f"lora_{normalized_character_tag}.csv")
    lora_dataset.to_csv(lora_dataset_output_file, index=False)
    print(f"LORA dataset for {character_tag} saved to {lora_dataset_output_file}")
    
    return lora_dataset

# Test it with John Egbert
john_egbert_lora_dataset = create_lora_dataset_for_character_tag(four_acts_hs_df, "John Egbert")
john_egbert_lora_dataset.sample(5)

LORA dataset for John Egbert saved to ../output\lora_johnegbert.csv


,image,caption
2643,https://homestuck.com/images/storyfiles/hs2/00...,What is left of the SPRITE undergoes a mysteri...
3621,https://homestuck.com/images/storyfiles/hs2/01...,You make a deadly BETTY CROCKER BARBASOL BOMB....
2925,https://homestuck.com/images/storyfiles/hs2/00...,Let's see how they like the old doublebarrel l...
2811,https://homestuck.com/images/storyfiles/hs2/00...,<NA>
2428,https://homestuck.com/images/storyfiles/hs2/00...,"On the desk is a DECK OF PLAYING CARDS, one of..."


In [61]:
# TODO: Now we can loop through the character tags and create a LORA dataset for each one
pass

In [62]:
# Now let's make style LORAs, though we already know which tags we want, sprite mode, hero mode, lined mode, and scribble mode (TODO: homosuck and hussnasty)
# We'll just filter the dataset for each style and create a LORA dataset for it

# First, let's get the tags for the styles
style_tags = ["Sprite Mode", "Hero Mode", "Lined Mode", "Scribble Mode"]

def create_lora_dataset_for_style_tag(df: pd.DataFrame, style_tag: str) -> pd.DataFrame:
    """
    Creates a LORA dataset for a specific style tag.

    Args:
        df (pd.DataFrame): The dataset to create the LORA dataset from.
        style_tag (str): The style tag to create the LORA dataset for.

    Returns:
        pd.DataFrame: The LORA dataset for the style tag.
    """
    # Get the normalized style tag
    normalized_style_tag = style_tag.lower().replace(" ", "")

    # Filter the dataset for the style tag
    style_df = df[df['tags'].apply(lambda tags: style_tag in tags)]
    
    # Filter for png / jpg / jpeg / gif files
    style_df = style_df[style_df['src'].apply(lambda src: src.endswith(('.png', '.jpg', '.jpeg', '.gif')))]

    # Create the LORA dataset
    # TODO: once again, we should have an actual caption here
    lora_dataset = style_df[['src', 'content']].copy()
    lora_dataset['src'] = lora_dataset['src'].apply(lambda src: f"https://homestuck.com/images{src}")
    
    # Save using the normalized style tag
    lora_dataset_output_file = os.path.join(OUTPUT_FOLDER, f"lora_{normalized_style_tag}.csv")
    lora_dataset.to_csv(lora_dataset_output_file, index=False)
    print(f"LORA dataset for {style_tag} saved to {lora_dataset_output_file}")
    
    return lora_dataset

# Test it with Sprite Mode
sprite_mode_lora_dataset = create_lora_dataset_for_style_tag(four_acts_hs_df, "Sprite Mode")
sprite_mode_lora_dataset.sample(5)

LORA dataset for Sprite Mode saved to ../output\lora_spritemode.csv


,src,content
3615,https://homestuck.com/images/storyfiles/hs2/01...,You make the REMOTE GHOST GAUNTLET. <br /><br ...
3136,https://homestuck.com/images/storyfiles/hs2/00...,What pumpkin?
4564,https://homestuck.com/images/storyfiles/hs2/01...,You upgrade the PUPPET TUX future Dave made. H...
3724,https://homestuck.com/images/storyfiles/hs2/01...,Who is this John claiming to be your friend? A...
4487,https://homestuck.com/images/storyfiles/hs2/01...,Hoofprints in the sand. The mystery deepens.
